In [26]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [27]:
import pandas as pd
import numpy as np
import re
import math
import datetime

import warnings
warnings.filterwarnings('ignore')
# pd.options.display.float_format = '{:.5f}'.format
pd.reset_option('display.float_format')

In [28]:
df = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/fnc(1989~2021)_이익변수.csv') # 경로변경

df.columns = df.iloc[8]
df = df.iloc[10:].reset_index(drop=True)
df.columns = ['code','cor_name','Accounting_Closing','year','Annual','ROA','non_cur_assets','worth','retained_earnings','inven','cur_lia','cash_flow','interest','cash','sales','trade_receiv','trade_pay','sales_grow','assets','cur_assets','lia_ratio','lia', 'net_income_grow','gross_margin', 'non_cur_lia','net_income', 'EBITDA', 'gross_profit', 'operating_profit', 'retained_earning', 'closing_price','revised_price', 'market_capital']
df['year'] = df['year'].astype(str)
df

,code,cor_name,Accounting_Closing,year,Annual,ROA,non_cur_assets,worth,retained_earnings,inven,...,gross_margin,non_cur_lia,net_income,EBITDA,gross_profit,operating_profit,retained_earning,closing_price,revised_price,market_capital
0,A005930,삼성전자,12,1989,Annual,5.90,"1,831,380,618","689,695,969","322,132,912","587,919,381",...,"914,441,785","1,159,879,306","158,481,699","911,276,720","914,441,785","378,244,847","322,132,912","45,300",349,"1,788,210"
1,A005930,삼성전자,12,1990,Annual,2.10,"2,494,814,647","787,868,737","367,776,600","722,804,594",...,"1,047,541,215","1,615,222,074","73,019,201","952,684,578","1,047,541,215","425,987,798","367,776,600","33,000",429,"1,376,850"
2,A005930,삼성전자,12,1991,Annual,1.43,"3,502,351,113","968,238,345","403,091,888","912,351,201",...,"1,425,315,571","1,811,166,814","68,627,896","949,968,847","1,425,315,571","664,770,660","403,091,888","31,300",310,"1,479,532"
3,A005930,삼성전자,12,1992,Annual,1.22,"4,199,375,837","1,140,320,721","439,579,978","866,618,101",...,"1,661,199,061","2,183,072,430","72,438,300","1,223,523,083","1,661,199,061","834,166,980","439,579,978","33,300",313,"1,728,236"
4,A005930,삼성전자,12,1993,Annual,1.04,"5,609,299,427","1,741,834,403","566,219,454","1,079,523,174",...,"2,675,141,518","3,224,629,270","76,484,817","1,307,650,612","2,675,141,518","1,307,650,612","566,219,454","57,400",336,"3,329,269"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115396,A950180,SNK,7.0,2017,Annual,NaN,"7,674,635","63,011,277","27,595,082","44,710",...,"48,550,511","367,598","29,776,151","33,648,805","48,550,511","33,394,379","27,595,082",NaN,NaN,NaN
115397,A950180,SNK,7.0,2018,Annual,27.48,"5,509,987","87,603,929","52,525,572","1,921,066",...,"59,280,108","1,703,795","24,839,476","37,268,232","59,280,108","37,046,369","52,525,572",NaN,NaN,NaN
115398,A950180,SNK,7.0,2019,Annual,14.64,"15,621,087","303,591,825","89,039,852","648,030",...,"80,935,428","3,164,597","31,169,202","47,619,725","80,935,428","47,164,517","89,039,852","18,800","33,650","395,962"
115399,A950180,SNK,7.0,2020,Annual,6.70,"41,607,598","332,501,209","115,388,523","798,844",...,"72,779,906","4,877,048","22,653,190","28,048,902","72,779,906","23,918,863","115,388,523","13,400","18,550","282,228"


# #1. 전처리

## (1) 결측치 처리

In [29]:
## 결측치 확인
df.apply(lambda x: pd.isna(x).sum())

code                      0
cor_name                132
Accounting_Closing     1468
year                      0
Annual                    0
ROA                   57785
non_cur_assets        58589
worth                 54985
retained_earnings     54985
inven                 58589
cur_lia               58589
cash_flow             54980
interest              54980
cash                  54985
sales                 54980
trade_receiv          58589
trade_pay             58589
sales_grow            58283
assets                54985
cur_assets            58589
lia_ratio             55000
lia                   54985
net_income_grow       58034
gross_margin          54980
non_cur_lia           58589
net_income            54980
EBITDA                54993
gross_profit          54980
operating_profit      54980
retained_earning      54985
closing_price         67221
revised_price         67221
market_capital        67221
dtype: int64

In [30]:
## 모든 열에 대해 nan값인 행 삭제
df = df.dropna(axis=0, how = 'all')
df # 없음

,code,cor_name,Accounting_Closing,year,Annual,ROA,non_cur_assets,worth,retained_earnings,inven,...,gross_margin,non_cur_lia,net_income,EBITDA,gross_profit,operating_profit,retained_earning,closing_price,revised_price,market_capital
0,A005930,삼성전자,12,1989,Annual,5.90,"1,831,380,618","689,695,969","322,132,912","587,919,381",...,"914,441,785","1,159,879,306","158,481,699","911,276,720","914,441,785","378,244,847","322,132,912","45,300",349,"1,788,210"
1,A005930,삼성전자,12,1990,Annual,2.10,"2,494,814,647","787,868,737","367,776,600","722,804,594",...,"1,047,541,215","1,615,222,074","73,019,201","952,684,578","1,047,541,215","425,987,798","367,776,600","33,000",429,"1,376,850"
2,A005930,삼성전자,12,1991,Annual,1.43,"3,502,351,113","968,238,345","403,091,888","912,351,201",...,"1,425,315,571","1,811,166,814","68,627,896","949,968,847","1,425,315,571","664,770,660","403,091,888","31,300",310,"1,479,532"
3,A005930,삼성전자,12,1992,Annual,1.22,"4,199,375,837","1,140,320,721","439,579,978","866,618,101",...,"1,661,199,061","2,183,072,430","72,438,300","1,223,523,083","1,661,199,061","834,166,980","439,579,978","33,300",313,"1,728,236"
4,A005930,삼성전자,12,1993,Annual,1.04,"5,609,299,427","1,741,834,403","566,219,454","1,079,523,174",...,"2,675,141,518","3,224,629,270","76,484,817","1,307,650,612","2,675,141,518","1,307,650,612","566,219,454","57,400",336,"3,329,269"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115396,A950180,SNK,7.0,2017,Annual,NaN,"7,674,635","63,011,277","27,595,082","44,710",...,"48,550,511","367,598","29,776,151","33,648,805","48,550,511","33,394,379","27,595,082",NaN,NaN,NaN
115397,A950180,SNK,7.0,2018,Annual,27.48,"5,509,987","87,603,929","52,525,572","1,921,066",...,"59,280,108","1,703,795","24,839,476","37,268,232","59,280,108","37,046,369","52,525,572",NaN,NaN,NaN
115398,A950180,SNK,7.0,2019,Annual,14.64,"15,621,087","303,591,825","89,039,852","648,030",...,"80,935,428","3,164,597","31,169,202","47,619,725","80,935,428","47,164,517","89,039,852","18,800","33,650","395,962"
115399,A950180,SNK,7.0,2020,Annual,6.70,"41,607,598","332,501,209","115,388,523","798,844",...,"72,779,906","4,877,048","22,653,190","28,048,902","72,779,906","23,918,863","115,388,523","13,400","18,550","282,228"


## (2) string 데이터 처리

In [31]:
## string 타입 회계값 확인
sale_str = df['sales_grow']
income_str = df['net_income_grow']
df[df['sales_grow']=='적전'] # 숫자가 아닌 '적자전환'등으로 표현

,code,cor_name,Accounting_Closing,year,Annual,ROA,non_cur_assets,worth,retained_earnings,inven,...,gross_margin,non_cur_lia,net_income,EBITDA,gross_profit,operating_profit,retained_earning,closing_price,revised_price,market_capital
2551,A006800,미래에셋증권,3,1999,Annual,-25.93,NaN,"734,792,199","-1,605,597",NaN,...,"-138,007,981",NaN,"-1,232,667,183","-1,226,426,037","-138,007,981","-1,255,332,076","-1,605,597","5,980","13,030","957,244"
2755,A029780,삼성카드,12,2005,Annual,-9.35,NaN,"1,462,089,501","-2,235,155,292",NaN,...,"-808,962,842",NaN,"-1,301,250,633","-1,195,208,985","-808,962,842","-1,376,719,984","-2,235,155,292",NaN,NaN,NaN
3077,A005940,NH투자증권,3,1997,Annual,-9.12,NaN,"715,270,372","24,137,851",NaN,...,"-20,941,212",NaN,"-189,634,737","-214,509,698","-20,941,212","-229,782,825","24,137,851","5,600","5,439","266,092"
3440,A008560,메리츠증권,3,1997,Annual,-25.83,NaN,"46,047,251","-55,011,889",NaN,...,"-57,988,768",NaN,"-90,057,276","-96,122,863","-57,988,768","-100,511,349","-55,011,889","1,920",599,"35,332"
12218,A003530,한화투자증권,3,1997,Annual,-8.06,NaN,"163,007,704","-2,459,332",NaN,...,"-58,891,133",NaN,"-127,108,066","-170,110,003","-58,891,133","-175,233,462","-2,459,332","1,950","2,519","33,536"
12237,A003530,한화투자증권,12,2016,Annual,-2.20,NaN,"815,070,240","-31,978,215",NaN,...,"-7,538,471",NaN,"-160,783,574","-175,168,688","-7,538,471","-192,880,065","-31,978,215","2,085","3,270","368,471"
12679,A003470,유안타증권,3,1996,Annual,-10.22,NaN,"251,443,314","16,898,797",NaN,...,"-25,801,279",NaN,"-123,917,954","-131,379,154","-25,801,279","-140,497,051","16,898,797","5,150","9,452","113,362"
15754,A041190,우리기술투자,12,2002,Annual,-6.52,NaN,"47,382,701","24,359,744",NaN,...,"-1,634,523",NaN,"-4,153,286","-3,999,193","-1,634,523","-4,088,413","24,359,744","2,840",451,"11,360"
15756,A041190,우리기술투자,12,2004,Annual,-15.79,NaN,"43,724,206","16,163,806",NaN,...,"-7,033,999",NaN,"-9,006,766","-9,160,355","-7,033,999","-9,262,850","16,163,806",555,471,"22,200"
15760,A041190,우리기술투자,12,2008,Annual,-8.40,NaN,"57,121,878","12,669,951",NaN,...,"-5,200,496",NaN,"-9,899,969","-8,510,226","-5,200,496","-9,226,081","12,669,951",245,775,"20,580"


- 흑전 = "흑자전환"

- 적전 = "적자전환"

- 적지 = "적자지속"

In [32]:
## 모든 데이터 str타입을 float타입으로 변환
for i in df.columns[5:]:
  df[i] = df[i].astype('str')
  df[i]=[re.sub(',','',df[i][j]) for j in range(len(df))]
  df[i] = df[i].apply(pd.to_numeric, errors='coerce') # '적지','적전','흑전'은 nan값으로 변환됨

In [33]:
## '적지','적전','흑전' -> 직접 매출액증가율, 당기순이익증가율 계산해 채워넣기
df['sale_str'] = sale_str
df['income_str'] = income_str
condition1 = (df['sale_str']=='흑전')|(df['sale_str']=='적전')|(df['sale_str']=='적지')
condition2 = (df['income_str']=='흑전')|(df['income_str']=='적전')|(df['income_str']=='적지')

  #매출증가율 변환
for i in df[condition1].index:
  if df[condition1].loc[i]['year'] == '1989': #1989년 이전 시점은 필요없으므로 제외
    pass
  else:
    int_sales = (df['sales'].loc[i] - df['sales'].loc[i-1])/df['sales'].loc[i-1] #증가율 계산 = (현재시점-이전시점)/(이전시점)
    df['sales_grow'][i] = float(int_sales)

  #당기순이익증가율 변환
for j in df[condition2].index:
  if df[condition2].loc[j]['year'] == '1989': #1989년 이전 시점은 필요없으므로 제외
    pass
  else:
    int_sales = (df['net_income'].loc[j] - df['net_income'].loc[j-1])/df['net_income'].loc[j-1] #증가율 계산 = (현재시점-이전시점)/(이전시점)
    df['net_income_grow'][j] = float(int_sales)

In [34]:
# 변환 결과

# 계산이 잘 되었는지 아래 결과와 비교해서 확인
print(df.loc[7:9][['year','net_income','income_str', 'net_income_grow']])
print((-938296384.0 - (-79508935.0)) / -79508935.0)

df[condition2][['income_str', 'net_income_grow', 'year']] #sales_grow값이 string값 또는 nan값이 아닌 숫자로 채워짐을 알 수 있다.
# 흑자전환인데 성장률이 음수인 경우가 가능한 게 맞는지 여쭈어보기. 가능하지..!

   year   net_income income_str  net_income_grow
7  1996  -79508935.0         적전        -1.037631
8  1997 -938296384.0         적지        10.801144
9  1998 -412783971.0         적지        -0.560071
10.80114390917197


,income_str,net_income_grow,year
7,적전,-1.037631,1996
8,적지,10.801144,1997
9,적지,-0.560071,1998
10,흑전,-8.717602,1999
65,흑전,-3.058273,2021
...,...,...,...
115290,적지,3.281130,2010
115323,적전,-2.396086,2010
115324,적지,-0.619974,2011
115325,적지,-0.584084,2012


In [35]:
df = df.drop(['sale_str',	'income_str'],axis=1)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115401 entries, 0 to 115400
Data columns (total 33 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   code                115401 non-null  object 
 1   cor_name            115269 non-null  object 
 2   Accounting_Closing  113933 non-null  object 
 3   year                115401 non-null  object 
 4   Annual              115401 non-null  object 
 5   ROA                 55370 non-null   float64
 6   non_cur_assets      56812 non-null   float64
 7   worth               60416 non-null   float64
 8   retained_earnings   60416 non-null   float64
 9   inven               56812 non-null   float64
 10  cur_lia             56812 non-null   float64
 11  cash_flow           60421 non-null   float64
 12  interest            60421 non-null   float64
 13  cash                60416 non-null   float64
 14  sales               60421 non-null   float64
 15  trade_receiv        56812 non-null

# #2. 재무비율

- 직접 계산할 변수
  - Cash and Marketable Securities / Assets
  - (변화율)Change in ROA => 어떻게 산출되는건지는 알 수 없음 @@
  - Liabilities / Assets
  - Total Liabilities less Cash to Total Assets = {총부채 - 현금} / 총자산
  - Current Liabilities / Sales
  - Current Assests / Current Liabilities
  - Retained Earning / Current Liabilities
  - LTD to {LTD plus Net worth} = 비유동부채 / {비유동부채 + 자본}
  - Cash Flow / Interest Expense
  - EBITDA to Interest Expense = EBITDA / 이자비용
  - Inventory / Sales
  - (변화율)Change in AR Turnover = 매출채권회전율 변화 = 매출액/매출채권의 변화 @@
  - Net P&L / Turnover =>>> 매출액순이익률 = 당기순이익 / 매출액 @@
  - (변화율)Change in Trade Debtors to Turnover {Change in Accounts Receivable to Sales} = 매입채무회전율 변화 = 매출액/매입채무의 변화
  - Gross Profit to Total Assets = 매출총이익 / 총자산
  - Retained Earnings to Net Sales = 이익잉여금 / 순매출액

- FnGuide 기존 변수
  - Total Assets
  - Real Net Sales
  - ROA = 순이익 / 총자산
  - (변화율)Sales Growth = Turnover {Sales} Growth
  - (변화율)Previous Year's Net Income to Previous Year's Net Sales

## (1) 직접 계산할 변수 - 다른 항목과의 비율

In [36]:
# panel = df[['code', 'cor_name', 'Accounting_Closing', 'year']]

# # (1) Cash and Marketable Securities / Assets
# panel['cash/assets'] = df['cash']/df['assets']
# # (2) Liabilities / Assets
# panel['liabilities/assets'] = df['lia']/df['assets']
# # (3) Total Liabilities less Cash to Total Assets = {총부채 - 현금} / 총자산
# panel['liabilities-cash/assets'] = (df['lia']-df['cash'])/df['assets']
# # (4) Current Liabilities / Sales
# panel['cur_liabilities/sales'] = df['cur_lia']/df['sales']
# # (5) Current Assests / Current Liabilities
# panel['cur_assets/cur_liabilities'] = df['cur_assets']/df['cur_lia']
# # (6) Retained Earning / Current Liabilities
# panel['retained_earnings/cur_liabilities'] = df['retained_earnings']/df['cur_lia']
# # (7) LTD to {LTD plus Net worth} = 비유동부채 / {비유동부채 + 자본}
# panel['non_cur_lia/non_cur_lia+worth'] = df['non_cur_lia']/(df['non_cur_lia']+df['worth'])
# # (8) Cash Flow / Interest Expense
# panel['interest/cash_flow'] = df['interest']/df['cash_flow']
# # (9) EBITDA to Interest Expense = EBITDA / 이자비용
# panel['interest/ebitda'] = df['interest']/df['EBITDA']
# # (10) Inventory / Sales
# panel['inven/sales'] = df['inven']/df['sales']
# # (11-1) AR Turnover -> 매출채권회전율 변화 = 매출액/매출채권의 변화
# panel['sales/trade_receiv'] = df['sales']/df['trade_receiv']
# # (12) Net P&L / Turnover = 매출액순이익률 = 당기순이익 / 매출액
# panel['net/sales'] = df['net_income']/df['sales']
# # (13-1) Change in Trade Debtors to Turnover {Change in Accounts Receivable to Sales} = 매입채무회전율 변화 = 매출액/매입채무의 변화
# panel['sales/trade_pay'] = df['sales']/df['trade_pay']
# # (14) Gross Profit to Total Assets = 매출총이익 / 총자산
# panel['gross_margin/sales'] = df['gross_margin']/df['sales']
# # (15) Retained Earnings to Net Sales = 이익잉여금 / 순매출액
# panel['retained_earnings/sales'] = df['retained_earnings']/df['sales']

- 분모분자 스위치
  - (8) Cash Flow / Interest Expense -> **Interest Expense / Cash Flow**
  - (9) EBITDA / Interest Expense -> **Interest Expense / EBITDA**

In [37]:
panel = df[['code', 'cor_name', 'Accounting_Closing', 'year']]

# (1) Cash and Marketable Securities / Assets
panel['cash/assets'] = df['cash']/df['assets']
# (2) Liabilities / Assets
panel['liabilities/assets'] = df['lia']/df['assets']
# (3) Total Liabilities less Cash to Total Assets = {총부채 - 현금} / 총자산
panel['liabilities-cash/assets'] = (df['lia']-df['cash'])/df['assets']
# (4)$$ Current Liabilities / Sales
panel['sales/cur_liabilities'] = df['sales']/df['cur_lia']
# (5) Current Assests / Current Liabilities
panel['cur_assets/cur_liabilities'] = df['cur_assets']/df['cur_lia']
# (6) Retained Earning / Current Liabilities
panel['retained_earnings/cur_liabilities'] = df['retained_earnings']/df['cur_lia']
# (7) LTD to {LTD plus Net worth} = 비유동부채 / {비유동부채 + 자본}
panel['non_cur_lia/non_cur_lia+worth'] = df['non_cur_lia']/(df['non_cur_lia']+df['worth'])
# (8)$$ Cash Flow / Interest Expense
panel['interest/cash_flow'] = df['interest']/df['cash_flow']
# (9)$$ EBITDA to Interest Expense = EBITDA / 이자비용
panel['interest/ebitda'] = df['interest']/df['EBITDA']
# (10) Inventory / Sales
panel['inven/sales'] = df['inven']/df['sales']
# (11-1)$$ AR Turnover -> 매출채권회전율 변화 = 매출액/매출채권의 변화
panel['trade_receiv/sales'] = df['trade_receiv']/df['sales']
# (12)$$ Net P&L / Turnover = 매출액순이익률 = 당기순이익 / 매출액
panel['sales/net_income'] = df['sales']/df['net_income']
# (13-1)$$ Change in Trade Debtors to Turnover {Change in Accounts Receivable to Sales} = 매입채무회전율 변화 = 매출액/매입채무의 변화
panel['trade_pay/sales'] = df['trade_pay']/df['sales']
# (14) Gross Profit to Total Assets = 매출총이익 / 총자산
panel['gross_margin/sales'] = df['gross_margin']/df['sales']
# (15)$$ Retained Earnings to Net Sales = 이익잉여금 / 순매출액
panel['sales/retained_earnings'] = df['sales']/df['retained_earnings']

In [38]:
## 무한대(inf)값 개수 확인 후 (분모분자 스위치 이후)
panel.iloc[:,4:].apply(lambda x: np.isinf(x).sum())

cash/assets                            0
liabilities/assets                     1
liabilities-cash/assets                1
sales/cur_liabilities                  1
cur_assets/cur_liabilities             2
retained_earnings/cur_liabilities      1
non_cur_lia/non_cur_lia+worth          0
interest/cash_flow                    11
interest/ebitda                       11
inven/sales                           37
trade_receiv/sales                    33
sales/net_income                      38
trade_pay/sales                       59
gross_margin/sales                    17
sales/retained_earnings              384
dtype: int64

In [39]:
panel

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,non_cur_lia/non_cur_lia+worth,interest/cash_flow,interest/ebitda,inven/sales,trade_receiv/sales,sales/net_income,trade_pay/sales,gross_margin/sales,sales/retained_earnings
0,A005930,삼성전자,12,1989,0.018574,0.762954,0.744379,3.816037,1.026828,0.306796,0.627106,0.000079,0.000060,0.146730,0.083046,25.282459,0.084071,0.228222,12.438366
1,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,0.672144,0.000012,0.000008,0.160206,0.114296,61.788109,0.090710,0.232182,12.267552
2,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,0.651638,0.000000,0.000000,0.174542,0.139126,76.166253,0.112442,0.272677,12.967588
3,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,0.656881,0.001486,0.000561,0.142004,0.120317,84.247871,0.090547,0.272204,13.883191
4,A005930,삼성전자,12,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,0.649281,0.000000,0.000000,0.124302,0.104952,113.547603,0.052545,0.308030,15.337989
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115396,A950180,SNK,7.0,2017,0.611887,0.168936,-0.442952,4.380337,5.477438,2.218058,0.005800,0.020936,0.018697,0.000820,0.359629,1.830197,0.000388,0.890897,1.974852
115397,A950180,SNK,7.0,2018,0.527099,0.165288,-0.361812,4.341584,6.356767,3.357695,0.019078,0.000000,0.000000,0.028286,0.369634,2.734232,0.000720,0.872833,1.293025
115398,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,0.010316,0.000000,0.000000,0.006349,0.291340,3.274688,0.000754,0.792945,1.146334
115399,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,0.014456,0.006268,0.005792,0.009289,0.237914,3.796404,0.000994,0.846271,0.745314


## (2) 직접 계산할 변수 - 변화율 항목

In [40]:
# (16) Change in ROA
ROA_list = []
for i in df.index:
  if df.loc[i]['year'] == '1989': #1989년 이전 시점은 필요없으므로 제외
    change_roa = np.nan
    ROA_list.append(change_roa)
  else:
    change_roa = (df['ROA'].loc[i] - df['ROA'].loc[i-1])/df['ROA'].loc[i-1] #증가율 계산 = (현재시점-이전시점)/(이전시점)
    ROA_list.append(change_roa)

# (11-2) AR Turnover -> 매출채권회전율 변화 = 매출액/매출채권의 변화
AR_list = []
for i in df.index:
  if df.loc[i]['year'] == '1989': #1989년 이전 시점은 필요없으므로 제외
    change_AR = np.nan
    AR_list.append(change_AR)
  else:
    change_AR = (panel['trade_receiv/sales'].loc[i] - panel['trade_receiv/sales'].loc[i-1])/panel['trade_receiv/sales'].loc[i-1] #증가율 계산 = (현재시점-이전시점)/(이전시점)
    AR_list.append(change_AR)

# (13-2) Change in Trade Debtors to Turnover {Change in Accounts Receivable to Sales} = 매입채무회전율 변화 = 매출액/매입채무의 변화
TP_list = []
for i in df.index:
  if df.loc[i]['year'] == '1989': #1989년 이전 시점은 필요없으므로 제외
    change_TP = np.nan
    TP_list.append(change_TP)
  else:
    change_TP = (panel['trade_pay/sales'].loc[i] - panel['trade_pay/sales'].loc[i-1])/panel['trade_pay/sales'].loc[i-1] #증가율 계산 = (현재시점-이전시점)/(이전시점)
    TP_list.append(change_TP)

In [41]:
print(len(ROA_list))
print(len(AR_list))
print(len(TP_list))
print(len(panel))
print(len(df))

115401
115401
115401
115401
115401


In [42]:
panel['change_in_roa'] = ROA_list
panel['change_in_AR'] = AR_list
panel['change_in_TP'] = TP_list

In [43]:
panel

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,interest/ebitda,inven/sales,trade_receiv/sales,sales/net_income,trade_pay/sales,gross_margin/sales,sales/retained_earnings,change_in_roa,change_in_AR,change_in_TP
0,A005930,삼성전자,12,1989,0.018574,0.762954,0.744379,3.816037,1.026828,0.306796,...,0.000060,0.146730,0.083046,25.282459,0.084071,0.228222,12.438366,NaN,NaN,NaN
1,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,0.000008,0.160206,0.114296,61.788109,0.090710,0.232182,12.267552,-0.644068,0.376302,0.078972
2,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,0.000000,0.174542,0.139126,76.166253,0.112442,0.272677,12.967588,-0.319048,0.217245,0.239576
3,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,0.000561,0.142004,0.120317,84.247871,0.090547,0.272204,13.883191,-0.146853,-0.135195,-0.194717
4,A005930,삼성전자,12,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,...,0.000000,0.124302,0.104952,113.547603,0.052545,0.308030,15.337989,-0.147541,-0.127702,-0.419701
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115396,A950180,SNK,7.0,2017,0.611887,0.168936,-0.442952,4.380337,5.477438,2.218058,...,0.018697,0.000820,0.359629,1.830197,0.000388,0.890897,1.974852,NaN,NaN,NaN
115397,A950180,SNK,7.0,2018,0.527099,0.165288,-0.361812,4.341584,6.356767,3.357695,...,0.000000,0.028286,0.369634,2.734232,0.000720,0.872833,1.293025,NaN,0.027818,0.856271
115398,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,0.000000,0.006349,0.291340,3.274688,0.000754,0.792945,1.146334,-0.467249,-0.211815,0.047309
115399,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,0.005792,0.009289,0.237914,3.796404,0.000994,0.846271,0.745314,-0.542350,-0.183378,0.318245


## (3) FnGuide 기존 변수

  - Total Assets
  - Real Net Sales
  - ROA = 순이익 / 총자산
  - Sales Growth = Turnover {Sales} Growth
  - Previous Year's Net Income to Previous Year's Net Sales

In [44]:
df[['assets', 'sales', 'ROA', 'lia_ratio', 'sales_grow', 'net_income_grow','gross_profit', 'operating_profit', 'retained_earning', 'closing_price', 'revised_price', 'market_capital']].apply(lambda x: np.isinf(x).sum())

assets              0
sales               0
ROA                 0
lia_ratio           0
sales_grow          0
net_income_grow     0
gross_profit        0
operating_profit    0
retained_earning    0
closing_price       0
revised_price       0
market_capital      0
dtype: int64

In [45]:
# (17)
panel['assets'] = df['assets']
# (18) 매출액
panel['sales'] = df['sales']
# (19)
panel['roa'] = df['ROA']
# (20) 기존 재무비율 데이터 vs 직접계산
panel['lia_ratio'] = df['lia_ratio']
# (21)
panel['sales_grow'] = df['sales_grow']
# (22) 당기순이익
panel['net_income'] = df['net_income']
# (23)
panel['net_income_grow'] = df['net_income_grow']
# (24) 매출총이익
panel['gross_profit'] = df['gross_profit']
# (25) 영업이익
panel['operating_profit'] = df['operating_profit']
# (26) 잉여이익금
panel['retained_earning'] = df['retained_earning']
# (27) 종가
panel['closing_price'] = df['closing_price']
# (28) 수정주가
panel['revised_price'] = df['revised_price']
# (29) 시가총액
panel['market_capital'] = df['market_capital']

In [46]:
# (20) 기존 재무비율 데이터 vs 직접계산 비교
print(panel[['liabilities/assets','lia_ratio']].apply(lambda x: pd.isna(x).sum()))

# 필요없는 열 삭제
panel = panel.drop(['lia_ratio', 'trade_receiv/sales', 'trade_pay/sales'], axis=1)
len(panel.iloc[:,4:].columns)

liabilities/assets    54986
lia_ratio             55000
dtype: int64


28

In [47]:
## nan값 개수 확인
panel.iloc[:,4:].apply(lambda x: pd.isna(x).sum())

cash/assets                          54987
liabilities/assets                   54986
liabilities-cash/assets              54986
sales/cur_liabilities                58590
cur_assets/cur_liabilities           58589
retained_earnings/cur_liabilities    58590
non_cur_lia/non_cur_lia+worth        58590
interest/cash_flow                   55072
interest/ebitda                      55089
inven/sales                          58742
sales/net_income                     55148
gross_margin/sales                   55560
sales/retained_earnings              55150
change_in_roa                        65787
change_in_AR                         62169
change_in_TP                         63621
assets                               54985
sales                                54980
roa                                  60031
sales_grow                           60520
net_income                           54980
net_income_grow                      60326
gross_profit                         54980
operating_p

In [48]:
## 무한대(inf)값 개수 확인
panel.iloc[:,4:].apply(lambda x: np.isinf(x).sum())

cash/assets                            0
liabilities/assets                     1
liabilities-cash/assets                1
sales/cur_liabilities                  1
cur_assets/cur_liabilities             2
retained_earnings/cur_liabilities      1
non_cur_lia/non_cur_lia+worth          0
interest/cash_flow                    11
interest/ebitda                       11
inven/sales                           37
sales/net_income                      38
gross_margin/sales                    17
sales/retained_earnings              384
change_in_roa                         45
change_in_AR                         182
change_in_TP                         423
assets                                 0
sales                                  0
roa                                    0
sales_grow                             0
net_income                             0
net_income_grow                        0
gross_profit                           0
operating_profit                       0
retained_earning

## (4) 재무비율 전처리

In [49]:
## 1989년 데이터 drop
panel = panel[panel['year']!='1989'].reset_index(drop=True)
panel

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,roa,sales_grow,net_income,net_income_grow,gross_profit,operating_profit,retained_earning,closing_price,revised_price,market_capital
0,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,2.10,12.60,7.301920e+07,-53.930000,1.047541e+09,4.259878e+08,3.677766e+08,33000.0,429.0,1376850.0
1,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,1.43,15.86,6.862790e+07,-6.010000,1.425316e+09,6.647707e+08,4.030919e+08,31300.0,310.0,1479532.0
2,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,1.22,16.75,7.243830e+07,5.550000,1.661199e+09,8.341670e+08,4.395800e+08,33300.0,313.0,1728236.0
3,A005930,삼성전자,12,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,...,1.04,42.31,7.648482e+07,5.590000,2.675142e+09,1.307651e+09,5.662195e+08,57400.0,336.0,3329269.0
4,A005930,삼성전자,12,1994,0.022999,0.692778,0.669779,3.189266,1.128719,0.412265,...,11.11,35.44,1.002286e+09,1210.440000,4.414842e+09,2.635296e+09,1.520553e+09,109000.0,589.0,6715822.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111899,A950180,SNK,7.0,2017,0.611887,0.168936,-0.442952,4.380337,5.477438,2.218058,...,NaN,NaN,2.977615e+07,NaN,4.855051e+07,3.339438e+07,2.759508e+07,NaN,NaN,NaN
111900,A950180,SNK,7.0,2018,0.527099,0.165288,-0.361812,4.341584,6.356767,3.357695,...,27.48,24.63,2.483948e+07,-16.580000,5.928011e+07,3.704637e+07,5.252557e+07,NaN,NaN,NaN
111901,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,14.64,50.29,3.116920e+07,25.480000,8.093543e+07,4.716452e+07,8.903985e+07,18800.0,33650.0,395962.0
111902,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,6.70,-15.74,2.265319e+07,-27.320000,7.277991e+07,2.391886e+07,1.153885e+08,13400.0,18550.0,282228.0


# #3. 패널 자료에 데이터 추가

## (1) 일반 기업 정보 데이터 합치기

In [50]:
df_detail = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/finance_detail.csv') # 경로변경필수

df_detail.columns = df_detail.iloc[4]
df_detail = df_detail.iloc[5:].reset_index(drop=True).rename(columns={'Symbol':'code'})
df_detail.tail()

4,code,Name,상장폐지일자,한국표준산업분류10차(대분류),한국표준산업분류10차(세분류),한국표준산업분류10차(세세분류),한국표준산업분류10차(소분류),한국표준산업분류10차(중분류),한국표준산업분류코드10차(대분류),한국표준산업분류코드10차(세분류),한국표준산업분류코드10차(세세분류),한국표준산업분류코드10차(소분류),한국표준산업분류코드10차(중분류),주거래은행,최초상장일자,상장일자
3492,A950010,평산차업 KDR,20151105,"전문, 과학 및 기술 서비스업",회사 본부,제조업 회사 본부,회사 본부 및 경영 컨설팅 서비스업,전문 서비스업,M,7151,71511,715,71,NaN,20071126,20071126
3493,A950030,네프로아이티,20111107,정보통신업,그 외 기타 정보 서비스업,데이터베이스 및 온라인정보 제공업,기타 정보 서비스업,정보서비스업,J,6399,63991,639,63,신한은행,20090424,20090424
3494,A950070,중국고섬,20131004,"전문, 과학 및 기술 서비스업",회사 본부,제조업 회사 본부,회사 본부 및 경영 컨설팅 서비스업,전문 서비스업,M,7151,71511,715,71,NaN,20110125,20110125
3495,A950100,SBI모기지,20150107,금융 및 보험업,여신금융업,그 외 기타 여신금융업,기타 금융업,금융업,K,6491,64919,649,64,NaN,20120430,20120430
3496,A950180,SNK,20220518,정보통신업,게임 소프트웨어 개발 및 공급업,게임 소프트웨어 개발 및 공급업,소프트웨어 개발 및 공급업,출판업,J,5821,5821,582,58,미츠호은행,20190507,20190507


In [51]:
# 데이터 합치기
panel_df = pd.merge(panel,df_detail, how='outer',on='code')
panel_df.drop(['Name'], axis=1, inplace=True)
panel_df

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,한국표준산업분류10차(소분류),한국표준산업분류10차(중분류),한국표준산업분류코드10차(대분류),한국표준산업분류코드10차(세분류),한국표준산업분류코드10차(세세분류),한국표준산업분류코드10차(소분류),한국표준산업분류코드10차(중분류),주거래은행,최초상장일자,상장일자
0,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",C,264,264,264,26,우리은행,19750611,19750611
1,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",C,264,264,264,26,우리은행,19750611,19750611
2,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",C,264,264,264,26,우리은행,19750611,19750611
3,A005930,삼성전자,12,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,...,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",C,264,264,264,26,우리은행,19750611,19750611
4,A005930,삼성전자,12,1994,0.022999,0.692778,0.669779,3.189266,1.128719,0.412265,...,통신 및 방송 장비 제조업,"전자부품, 컴퓨터, 영상, 음향 및 통신장비 제조업",C,264,264,264,26,우리은행,19750611,19750611
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111899,A950180,SNK,7.0,2017,0.611887,0.168936,-0.442952,4.380337,5.477438,2.218058,...,소프트웨어 개발 및 공급업,출판업,J,5821,5821,582,58,미츠호은행,20190507,20190507
111900,A950180,SNK,7.0,2018,0.527099,0.165288,-0.361812,4.341584,6.356767,3.357695,...,소프트웨어 개발 및 공급업,출판업,J,5821,5821,582,58,미츠호은행,20190507,20190507
111901,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,소프트웨어 개발 및 공급업,출판업,J,5821,5821,582,58,미츠호은행,20190507,20190507
111902,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,소프트웨어 개발 및 공급업,출판업,J,5821,5821,582,58,미츠호은행,20190507,20190507


In [52]:
# 데이터 길이 동일
print(len(panel))
print(len(panel_df))

111904
111904


In [53]:
# 칼럼 수 확인
print(len(panel.columns))
print(len(df_detail.iloc[:,2:].columns))
print(len(panel_df.columns))

32
14
46


In [54]:
# 확인
panel_df[panel_df['cor_name']=='삼성전자'].iloc[:,:33]

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,sales_grow,net_income,net_income_grow,gross_profit,operating_profit,retained_earning,closing_price,revised_price,market_capital,상장폐지일자
0,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,12.60,7.301920e+07,-53.930000,1.047541e+09,4.259878e+08,3.677766e+08,33000.0,429.0,1376850.0,NaN
1,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,15.86,6.862790e+07,-6.010000,1.425316e+09,6.647707e+08,4.030919e+08,31300.0,310.0,1479532.0,NaN
2,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,16.75,7.243830e+07,5.550000,1.661199e+09,8.341670e+08,4.395800e+08,33300.0,313.0,1728236.0,NaN
3,A005930,삼성전자,12,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,...,42.31,7.648482e+07,5.590000,2.675142e+09,1.307651e+09,5.662195e+08,57400.0,336.0,3329269.0,NaN
4,A005930,삼성전자,12,1994,0.022999,0.692778,0.669779,3.189266,1.128719,0.412265,...,35.44,1.002286e+09,1210.440000,4.414842e+09,2.635296e+09,1.520553e+09,109000.0,589.0,6715822.0,NaN
5,A005930,삼성전자,12,1995,0.054644,0.693142,0.638497,2.801825,1.333899,0.514395,...,58.71,2.112871e+09,110.810000,7.272359e+09,4.022398e+09,3.427517e+09,141000.0,1113.0,11216897.0,NaN
6,A005930,삼성전자,12,1996,0.055147,0.777430,0.722283,2.550588,1.255358,0.424394,...,6.47,-7.950894e+07,-1.037631,4.946794e+09,1.371426e+09,3.307466e+09,45500.0,1896.0,4592777.0,NaN
7,A005930,삼성전자,12,1997,0.080563,0.854966,0.774403,1.548941,1.015839,0.180957,...,14.11,-9.382964e+08,10.801144,6.656275e+09,2.138743e+09,2.649855e+09,38400.0,783.0,3996910.0,NaN
8,A005930,삼성전자,12,1998,0.068267,0.789849,0.721582,2.756648,1.039684,0.237931,...,13.62,-4.127840e+08,-0.560071,7.451451e+09,2.706338e+09,2.224449e+09,80700.0,735.0,10918438.0,NaN
9,A005930,삼성전자,12,1999,0.015534,0.548482,0.532948,3.014123,0.985555,0.487541,...,24.50,3.185702e+09,-8.717602,1.061130e+10,5.375951e+09,5.190255e+09,266000.0,1693.0,44086252.0,NaN


## (2) 상장폐지 사유 데이터 추가

In [55]:
## 상장폐지 데이터
unlist = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/상장폐지.csv') # 경로변경
unlist

,번호,회사명,종목코드,폐지일자,폐지사유,비고
0,1699,금성전기,29260,1991-06-13,해산 사유 발생,NaN
1,1698,삼화,28440,1991-07-12,감사의견 의견거절,NaN
2,1697,금성투자금융,28450,1991-09-02,해산 사유 발생,NaN
3,1696,태평양건설,28460,1991-10-05,영업활동정지 6월 계속,NaN
4,1695,럭키소재,34370,1991-11-11,해산 사유 발생,NaN
...,...,...,...,...,...,...
1694,5,케이프이에스제4호,347140,2022-10-31,상장예비심사 청구서 미제출로 관리종목 지정 후 1개월 이내 동 사유 미해소,NaN
1695,4,LX세미콘,108320,2022-11-03,유가증권시장 상장,LX세미콘
1696,3,레드로버,60300,2022-11-04,기업의 계속성 및 경영의 투명성 등을 종합적을 고려하여 상장폐지기준에 해당한다고 결정,NaN
1697,2,참존글로벌,158310,2022-11-09,기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정,NaN


In [56]:
## 패널 자료 상장폐지 일자 & 기업명
panel_df['unlist_date'] = pd.to_datetime(panel_df['상장폐지일자']).astype(str)
panel_df = panel_df.replace({'unlist_date':'NaT'},np.nan)
panel_df['int_code'] = panel_df['code'].str.slice(start=1).astype(int)

panel_df[panel_df['상장폐지일자'].notnull()][['int_code', 'code', 'cor_name', 'unlist_date']] #int_code, unlist_date로 새로운 데이터와 통일

,int_code,code,cor_name,unlist_date
76640,10,A000010,조흥은행,2004-07-02
76641,10,A000010,조흥은행,2004-07-02
76642,10,A000010,조흥은행,2004-07-02
76643,10,A000010,조흥은행,2004-07-02
76644,10,A000010,조흥은행,2004-07-02
...,...,...,...,...
111899,950180,A950180,SNK,2022-05-18
111900,950180,A950180,SNK,2022-05-18
111901,950180,A950180,SNK,2022-05-18
111902,950180,A950180,SNK,2022-05-18


In [57]:
## 두번 상장폐지된 기업 있는지 확인
unlist_cor_date = panel_df[panel_df['상장폐지일자'].notnull()][['int_code', 'unlist_date']].drop_duplicates()
set(unlist_cor_date.groupby('int_code')['unlist_date'].count().values) #없음

{1}

In [58]:
print(len(unlist_cor_date))
print(len(panel_df[panel_df['상장폐지일자'].notnull()]['int_code'].unique()))

1082
1082


In [59]:
## 차집합 확인
print('종목코드 고유값 차집합 개수:',len(set(unlist['종목코드'].values) - set(unlist_cor_date['int_code'].values)))
print('unlist 고유값 개수:',len(set(unlist['종목코드'].values))) #고유값 개수
print('panel 고유값 개수:',len(set(unlist_cor_date['int_code'].values)),'\n') #고유값 개수

print('폐지일자 고유값 차집합 개수:',len(set(unlist['폐지일자'].values) - set(unlist_cor_date['unlist_date'].values)))
print('unlist 고유값 개수:',len(set(unlist['폐지일자'].values))) #고유값 개수
print('panel 고유값 개수:',len(set(unlist_cor_date['unlist_date'].values))) #고유값 개수

종목코드 고유값 차집합 개수: 590
unlist 고유값 개수: 1672
panel 고유값 개수: 1082 

폐지일자 고유값 차집합 개수: 404
unlist 고유값 개수: 1102
panel 고유값 개수: 732


In [60]:
unlist_cor_list = unlist_cor_date
panel_df['unlist_reason'] = '0'

for inx in panel_df[panel_df['상장폐지일자'].notnull()]['int_code'].index:
  code = panel_df['int_code'][inx]
  reason = unlist[unlist['종목코드']==code]['폐지사유'].values[0]
  panel_df['unlist_reason'][inx] = reason

panel_df['unlist_reason']

0                          0
1                          0
2                          0
3                          0
4                          0
                 ...        
111899    상장폐지 신청('22.04.14)
111900    상장폐지 신청('22.04.14)
111901    상장폐지 신청('22.04.14)
111902    상장폐지 신청('22.04.14)
111903    상장폐지 신청('22.04.14)
Name: unlist_reason, Length: 111904, dtype: object

In [61]:
## 개수 확인: 상장폐지일자가 있는 기업에 대한 사유가 모두 채워짐
print('상장폐지 일자가 있는 기업:',len(panel_df[panel_df['상장폐지일자'].notnull()]))
print('상장폐지 사유 있는 기업 개수:',len(panel_df[panel_df['unlist_reason']!='0']))

## 확실한 확인!
# panel_df.loc[(panel_df['unlist_reason']!='0')&(panel_df['unlist_date'].isnull())][['int_code', 'unlist_date','unlist_reason']]
# panel_df.loc[(panel_df['unlist_reason']=='0')&(panel_df['unlist_date'].notnull())][['int_code', 'unlist_date','unlist_reason']]

상장폐지 일자가 있는 기업: 34624
상장폐지 사유 있는 기업 개수: 34624


In [62]:
## 칼럼 정리 (drop, 순서, 이름)
panel_df.drop(['int_code', '상장폐지일자'], axis=1, inplace=True)
panel_df = panel_df[['code', 'cor_name', 'Accounting_Closing', 'year', 'cash/assets', 'liabilities/assets', 'liabilities-cash/assets', 'sales/cur_liabilities', 'cur_assets/cur_liabilities','retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth', 'interest/cash_flow', 'interest/ebitda', 'inven/sales','sales/net_income', 'gross_margin/sales', 'sales/retained_earnings', 'change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales','roa', 'sales_grow', 'net_income', 'net_income_grow', 'gross_profit',	'operating_profit', 'retained_earning', 'closing_price','revised_price', 'market_capital','한국표준산업분류10차(대분류)','한국표준산업분류10차(세분류)', '한국표준산업분류10차(세세분류)', '한국표준산업분류10차(소분류)','한국표준산업분류10차(중분류)', '한국표준산업분류코드10차(대분류)', '한국표준산업분류코드10차(세분류)','한국표준산업분류코드10차(세세분류)', '한국표준산업분류코드10차(소분류)', '한국표준산업분류코드10차(중분류)','주거래은행', '최초상장일자', '상장일자', 'unlist_date', 'unlist_reason']]
panel_df.columns = ['code', 'cor_name', 'Accounting_Closing', 'year', 'cash/assets', 'liabilities/assets', 'liabilities-cash/assets', 'sales/cur_liabilities', 'cur_assets/cur_liabilities','retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth', 'interest/cash_flow', 'interest/ebitda', 'inven/sales','sales/net_income', 'gross_margin/sales', 'sales/retained_earnings', 'change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales','roa', 'sales_grow', 'net_income', 'net_income_grow', 'gross_profit',	'operating_profit', 'retained_earning', 'closing_price','revised_price', 'market_capital', '대분류','세분류', '세세분류', '소분류','중분류', '코드대분류', '코드세분류','코드세세분류', '코드소분류', '코드중분류','주거래은행', '최초상장일자', '상장일자', 'unlist_date', 'unlist_reason']

In [63]:
panel_df.tail(3)

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,코드대분류,코드세분류,코드세세분류,코드소분류,코드중분류,주거래은행,최초상장일자,상장일자,unlist_date,unlist_reason
111901,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)
111902,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)
111903,A950180,SNK,7.0,2021,0.694562,0.101859,-0.592702,2.830798,9.826040,1.322351,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)


In [64]:
panel_df['unlist_reason'].nunique()

269

## (3) 주가 데이터 추가

In [65]:
## 수익률 데이터 추가
stock_earning = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/stock_earning(1990~2021).csv')
stock_earning = stock_earning.iloc[7:].drop([9, 10, 11, 12], axis = 0).reset_index(drop=True)
stock_earning.head(5)

,Unnamed: 0,Last Refresh: 2022-11-26 16:28:28,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 3488,Unnamed: 3489,Unnamed: 3490,Unnamed: 3491,Unnamed: 3492,Unnamed: 3493,Unnamed: 3494,Unnamed: 3495,Unnamed: 3496,Unnamed: 3497
0,Symbol,A005930,A373220,A000660,A207940,A006400,A051910,A005380,A035420,A000270,...,A900080,A900090,A900130,A900150,A900180,A950010,A950030,A950070,A950100,A950180
1,Symbol Name,삼성전자,LG에너지솔루션,SK하이닉스,삼성바이오로직스,삼성SDI,LG화학,현대차,NAVER,기아,...,에스앤씨엔진그룹,차이나하오란,웨이포트,성융광전투자,완리,평산차업 KDR,네프로아이티,중국고섬,SBI모기지,SNK
2,1990-12-31,-26.50,NaN,NaN,NaN,-34.31,NaN,-26.71,NaN,-29.96,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1991-12-31,-4.08,NaN,NaN,NaN,21.28,NaN,29.90,NaN,16.39,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1992-12-31,8.17,NaN,NaN,NaN,6.59,NaN,-11.87,NaN,-17.56,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
stock_return = pd.DataFrame()

## code 패널화
stock_earning.columns = stock_earning.iloc[0]
stock_return = stock_earning.melt(id_vars=['Symbol'], var_name='code')[['Symbol', 'code']]

## cor_name 패널화
stock_earning.columns = stock_earning.iloc[1]
stock_earning = stock_earning.drop([0,1], axis=0).reset_index(drop=True)
stock_return[['cor_name', 'return']] = stock_earning.melt(id_vars=['Symbol Name'], var_name='cor_name', value_name='return')[['cor_name', 'return']]

stock_return = stock_return[(stock_return['Symbol']!='Symbol')&(stock_return['Symbol']!='Symbol Name')]
stock_return['Symbol'] = stock_return['Symbol'].str.slice(stop=4)
stock_return

,Symbol,code,cor_name,return
2,1990,A005930,삼성전자,8.17
3,1991,A005930,삼성전자,77.95
4,1992,A005930,삼성전자,94.40
5,1993,A005930,삼성전자,59.74
6,1994,A005930,삼성전자,-56.81
...,...,...,...,...
118893,2017,A950180,NaN,NaN
118894,2018,A950180,NaN,NaN
118895,2019,A950180,NaN,NaN
118896,2020,A950180,NaN,NaN


In [67]:
## shape, 변수 성분 일치 확인
print(panel_df.shape)
print(stock_return.shape)

print(len(set(panel_df['year'])))
print(len(set(stock_return['Symbol'])))

(111904, 47)
(111904, 4)
32
32


In [68]:
panel_df.columns

Index(['code', 'cor_name', 'Accounting_Closing', 'year', 'cash/assets',
       'liabilities/assets', 'liabilities-cash/assets',
       'sales/cur_liabilities', 'cur_assets/cur_liabilities',
       'retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth',
       'interest/cash_flow', 'interest/ebitda', 'inven/sales',
       'sales/net_income', 'gross_margin/sales', 'sales/retained_earnings',
       'change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales',
       'roa', 'sales_grow', 'net_income', 'net_income_grow', 'gross_profit',
       'operating_profit', 'retained_earning', 'closing_price',
       'revised_price', 'market_capital', '대분류', '세분류', '세세분류', '소분류', '중분류',
       '코드대분류', '코드세분류', '코드세세분류', '코드소분류', '코드중분류', '주거래은행', '최초상장일자', '상장일자',
       'unlist_date', 'unlist_reason'],
      dtype='object')

In [69]:
panel_df['return'] = stock_return['return']
panel_df = panel_df[['code', 'cor_name', 'Accounting_Closing', 'year', 'cash/assets','liabilities/assets', 'liabilities-cash/assets','sales/cur_liabilities', 'cur_assets/cur_liabilities','retained_earnings/cur_liabilities', 'non_cur_lia/non_cur_lia+worth','interest/cash_flow', 'interest/ebitda', 'inven/sales','sales/net_income', 'gross_margin/sales', 'sales/retained_earnings','change_in_roa', 'change_in_AR', 'change_in_TP', 'assets', 'sales','roa', 'sales_grow', 'net_income', 'net_income_grow', 'gross_profit',	'operating_profit', 'retained_earning', 'closing_price','revised_price', 'market_capital', 'return', '대분류', '세분류', '세세분류', '소분류', '중분류','코드대분류', '코드세분류', '코드세세분류', '코드소분류', '코드중분류', '주거래은행', '최초상장일자', '상장일자','unlist_date', 'unlist_reason']]
panel_df

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,코드대분류,코드세분류,코드세세분류,코드소분류,코드중분류,주거래은행,최초상장일자,상장일자,unlist_date,unlist_reason
0,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,C,264,264,264,26,우리은행,19750611,19750611,NaN,0
1,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,C,264,264,264,26,우리은행,19750611,19750611,NaN,0
2,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,C,264,264,264,26,우리은행,19750611,19750611,NaN,0
3,A005930,삼성전자,12,1993,0.033530,0.793795,0.760265,2.495139,0.815311,0.162677,...,C,264,264,264,26,우리은행,19750611,19750611,NaN,0
4,A005930,삼성전자,12,1994,0.022999,0.692778,0.669779,3.189266,1.128719,0.412265,...,C,264,264,264,26,우리은행,19750611,19750611,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111899,A950180,SNK,7.0,2017,0.611887,0.168936,-0.442952,4.380337,5.477438,2.218058,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)
111900,A950180,SNK,7.0,2018,0.527099,0.165288,-0.361812,4.341584,6.356767,3.357695,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)
111901,A950180,SNK,7.0,2019,0.817917,0.054197,-0.763720,7.171777,21.456253,6.256272,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)
111902,A950180,SNK,7.0,2020,0.600422,0.065299,-0.535123,4.686278,17.116893,6.287657,...,J,5821,5821,582,58,미츠호은행,20190507,20190507,2022-05-18,상장폐지 신청('22.04.14)


## (4) 기업코드&사업자번호 추가

In [70]:
business_code = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/기업명_법인_사업자.csv') # 경로변경
business_code.columns = ['Symbol',	'Name',	'non_name',	'사업자등록번호',	'법인등록번호']
business_code

,Symbol,Name,non_name,사업자등록번호,법인등록번호
0,A005930,삼성전자,삼성전자(주),124-81-00998,1.301110e+12
1,A373220,LG에너지솔루션,(주)엘지에너지솔루션,851-81-02050,1.101120e+12
2,A000660,SK하이닉스,에스케이하이닉스(주),126-81-03725,1.344110e+12
3,A207940,삼성바이오로직스,삼성바이오로직스(주),131-86-27632,1.201110e+12
4,A006400,삼성SDI,삼성SDI(주),124-81-31282,1.101110e+12
...,...,...,...,...,...
3492,A950010,평산차업 KDR,평산차업집단유한공사,000-81-10036,NaN
3493,A950030,네프로아이티,(주)네프로아이티,000-81-20003,NaN
3494,A950070,중국고섬,중국고섬공고유한공사,000-81-10039,NaN
3495,A950100,SBI모기지,SBI모기지(주),000-81-10040,NaN


In [71]:
## 딕셔너리화 {기업코드:사업자번호}
def df_to_dict(df, keys_col, values_col):

  trans = df.loc[:,[keys_col, values_col]].transpose()
  trans.columns = trans.iloc[0]
  trans = trans.iloc[1].to_dict()

  for k in list(trans.keys()):
    if pd.isnull(trans[k])==True:
      del trans[k]

  return trans

# firm_dict = df_to_dict(business_code, '법인등록번호')
bsn_dict = df_to_dict(business_code, 'Name','사업자등록번호')
print(bsn_dict)

{'삼성전자': '124-81-00998', 'LG에너지솔루션': '851-81-02050', 'SK하이닉스': '126-81-03725', '삼성바이오로직스': '131-86-27632', '삼성SDI': '124-81-31282', 'LG화학': '107-81-98139', '현대차': '101-81-09147', 'NAVER': '220-81-62517', '기아': '119-81-02316', '셀트리온': '133-81-23603', 'POSCO홀딩스': '506-81-00017', '삼성물산': '104-81-26728', '카카오': '120-81-47521', '현대모비스': '101-81-16406', 'KB금융': '201-86-08254', '신한지주': '104-81-64625', '포스코케미칼': '506-81-01452', 'SK이노베이션': '101-86-32319', 'SK': '116-81-02054', 'LG전자': '107-86-14075', '삼성생명': '104-81-26688', 'KT&G': '306-81-30866', 'LG': '116-81-18750', '고려아연': '211-81-11260', '하나금융지주': '107-86-71509', '한국전력': '120-82-00052', 'SK텔레콤': '104-81-37225', '카카오뱅크': '375-88-00197', '셀트리온헬스케어': '123-81-53473', '에코프로비엠': '638-86-00376', '삼성전기': '124-81-00979', '두산에너빌리티': '609-81-04684', 'S-Oil': '116-81-36743', 'HMM': '202-81-46197', '삼성에스디에스': '110-81-28774', '현대중공업': '252-87-01412', 'KT': '102-81-42945', '삼성화재': '202-81-45617', '한화솔루션': '202-81-16577', '크래프톤': '220-87-45316', '대한항공': '

In [72]:
## 사업자번호 & 법인등록번호 변환
# panel_df['firm_code'] = panel_df['code']
panel_df['bsn_code'] = panel_df['cor_name']

# panel_df = panel_df.replace({'firm_code':firm_dict})
panel_df = panel_df.replace({'bsn_code':bsn_dict})

panel_df.head(3)

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,코드세분류,코드세세분류,코드소분류,코드중분류,주거래은행,최초상장일자,상장일자,unlist_date,unlist_reason,bsn_code
0,A005930,삼성전자,12,1990,0.026243,0.805812,0.779569,2.738511,0.948362,0.223232,...,264,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998
1,A005930,삼성전자,12,1991,0.011561,0.825602,0.814041,1.887630,0.740129,0.145565,...,264,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998
2,A005930,삼성전자,12,1992,0.026253,0.819760,0.793507,2.032037,0.708325,0.146367,...,264,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998


## (5) 비정제 기업명 사전 & 변환

In [73]:
gas_2011 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2011년_온실가스배출량.csv')
gas_2012 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2012년_온실가스배출량.csv')
gas_2013 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2013년_온실가스배출량.csv')
gas_2014 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2014년_온실가스배출량.csv')
gas_2015 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2015년_온실가스배출량.csv')

gas_2021 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2021년_온실가스배출량.csv')

gas_2016 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2016 업체, 사업장 온실가스 배출량, 에너지 사용량_환경부 온실가스종합정보센터(UTF8) (법인등록번호,사업자등록번호).CSV') # 경로변경
gas_2017 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2017 업체, 사업장 온실가스 배출량, 에너지 사용량_환경부 온실가스종합정보센터(UTF8) (법인등록번호,사업자등록번호).CSV') # 경로변경
gas_2018 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2018 업체, 사업장 온실가스 배출량, 에너지 사용량_환경부 온실가스종합정보센터(UTF8) (법인등록번호,사업자등록번호).CSV') # 경로변경
gas_2019 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2019 업체, 사업장 온실가스 배출량, 에너지 사용량_환경부 온실가스종합정보센터(UTF8) (법인등록번호,사업자등록번호).CSV') # 경로변경

gas_2020 = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/온실가스데이터/2020 업체, 사업장 온실가스 배출량, 에너지 사용량_환경부 온실가스종합정보센터(법인등록번호,사업자등록번호).csv') # 경로변경

In [74]:
def gas_preprocessing(gas_df):

  if len(gas_df.columns) == 10:
    gas_df = gas_df.iloc[:,[2,3,4,5,6,7]]
    gas_df.columns = ['non_cor_name', 'year','ctgy','ctgy_detail','grngas','enrg']

  elif (len(gas_df.columns) == 19) | (len(gas_df.columns) == 20):
    # gas_df = gas_df.loc[:, :'ENRG_UNIT']
    gas_df = gas_df.iloc[:,[2,3,4,6,7,8,9,11]]
    gas_df.columns = ['non_cor_name','cor_code','bsn_code', 'year','ctgy','ctgy_detail','grngas','enrg']

  elif len(gas_df.columns) == 25:
    gas_df.columns = gas_df.iloc[0]
    gas_df = gas_df.iloc[1:, [2,3,4,6,7,8,9,11]].reset_index(drop=True)
    gas_df.columns = ['non_cor_name','cor_code','bsn_code', 'year','ctgy','ctgy_detail','grngas','enrg']

  else:
    gas_df = gas_df.iloc[:, [3,4,5,7,9,10]].reset_index(drop=True)
    gas_df.columns = ['non_cor_name', 'year','ctgy','ctgy_detail','grngas','enrg']

  return gas_df

gas_2011 = gas_preprocessing(gas_2011)
gas_2012 = gas_preprocessing(gas_2012)
gas_2013 = gas_preprocessing(gas_2013)
gas_2014 = gas_preprocessing(gas_2014)
gas_2015 = gas_preprocessing(gas_2015)
gas_2016 = gas_preprocessing(gas_2016) #법인등록 (유료)
gas_2017 = gas_preprocessing(gas_2017) #법인등록 (유료)
gas_2018 = gas_preprocessing(gas_2018) #법인등록 (유료)
gas_2019 = gas_preprocessing(gas_2019) #법인등록 (유료)
gas_2020 = gas_preprocessing(gas_2020) #법인등록 (유료)
gas_2021 = gas_preprocessing(gas_2021)

### a. 비정제 기업명 딕셔너리 사전 구축

- 사업자등록번호가 있는 연도데이터(2016~2020)를 사용해 비정제 기업명 딕셔너리 사전 구축

In [75]:
## 딕셔너리화 {사업자번호:cor_name}
name_bsn_dict = df_to_dict(business_code, '사업자등록번호', 'Name')
print(name_bsn_dict)

## 비정제 기업명-정제 기업명 dict 구축
def name_matching_dict(gas_df, dic):
  gas_df['cor'] = gas_df['bsn_code']

  gas_df = gas_df.replace({'cor':dic})

  gas_df['cor'] = gas_df['cor'].str.replace(pat=r"[0-9]+-[0-9]+-[0-9]+", repl= r'', regex=True)
  gas_df['cor'] = gas_df['cor'].replace(['-',''], np.nan)

  name_match_dict = df_to_dict(gas_df, 'non_cor_name', 'cor')

  return name_match_dict

name_match_dict_2016 = name_matching_dict(gas_2016,name_bsn_dict)
name_match_dict_2017 = name_matching_dict(gas_2017,name_bsn_dict)
name_match_dict_2018 = name_matching_dict(gas_2018,name_bsn_dict)
name_match_dict_2019 = name_matching_dict(gas_2019,name_bsn_dict)
name_match_dict_2020 = name_matching_dict(gas_2020,name_bsn_dict)
print(name_match_dict_2017)

{'124-81-00998': '삼성전자', '851-81-02050': 'LG에너지솔루션', '126-81-03725': 'SK하이닉스', '131-86-27632': '삼성바이오로직스', '124-81-31282': '삼성SDI', '107-81-98139': 'LG화학', '101-81-09147': '현대차', '220-81-62517': 'NAVER', '119-81-02316': '기아', '133-81-23603': '셀트리온', '506-81-00017': 'POSCO홀딩스', '202-81-45975': '삼성물산', '120-81-47521': '카카오', '101-81-16406': '현대모비스', '201-86-08254': 'KB금융', '104-81-64625': '신한지주', '506-81-01452': '포스코케미칼', '101-86-32319': 'SK이노베이션', '220-81-16503': 'SK', '107-86-14075': 'LG전자', '104-81-26688': '삼성생명', '306-81-30866': 'KT&G', '116-81-18750': 'LG', '211-81-11260': '고려아연', '107-86-71509': '하나금융지주', '120-82-00052': '한국전력', '104-81-37225': 'SK텔레콤', '375-88-00197': '카카오뱅크', '123-81-53473': '셀트리온헬스케어', '638-86-00376': '에코프로비엠', '124-81-00979': '삼성전기', '609-81-04684': '두산에너빌리티', '116-81-36743': 'S-Oil', '202-81-46197': 'HMM', '110-81-28774': '삼성에스디에스', '252-87-01412': '현대중공업', '102-81-42945': 'KT', '202-81-45617': '삼성화재', '202-81-16577': '한화솔루션', '220-87-45316': '크래프톤', '110-81-1

In [76]:
from itertools import chain

name_match_dict = dict()
for key, value in chain(name_match_dict_2017.items(),name_match_dict_2018.items(),name_match_dict_2019.items(),name_match_dict_2020.items()):
  if value in name_match_dict:
    name_match_dict[value].append(key)
    name_match_dict[value] = list(set(name_match_dict[value]))
  else:
    name_match_dict[value] = [key]

del name_match_dict_2016, name_match_dict_2017, name_match_dict_2018, name_match_dict_2019, name_match_dict_2020
# name_match_dict.pop(np.nan)

print(len(name_match_dict))
print(name_match_dict) # 여러개인가? 한국조선해양 검색

323
{'MH에탄올': ['(주)MH에탄올'], 'SIMPAC Metal': ['(주)SIMPAC METAL'], '강원랜드': ['(주)강원랜드'], '경방': ['(주)경방'], '국민은행': ['(주)국민은행'], '금비': ['(주)금비'], '대림제지': ['(주)대림제지'], '대우건설': ['(주)대우건설', '(주)대욱케스트'], '동방': ['(주)동방'], '동양고속': ['(주)동양고속'], '동원F&B': ['(주)동원F&B'], '두산': ['(주)두산'], '디에이피': ['(주)디에이피'], 'HL만도': ['(주)만도'], '태경비케이': ['주식회사 태경비케이', '(주)백광소재'], '벽산': ['(주)벽산'], '삼원강재': ['(주)삼원강재'], '삼천리': ['(주)삼천리'], '서원': ['(주)서원'], '서흥': ['(주)서흥'], '성광벤드': ['(주)성광벤드'], '세아베스틸지주': ['(주)세아베스틸'], '세아제강지주': ['(주)세아제강'], '세아특수강': ['(주)세아특수강'], '스타플렉스': ['(주)스타플렉스'], '신세계': ['(주)신세계'], '조흥은행': ['(주)신한은행'], '한국아트라스비엑스': ['(주)한국아트라스비엑스', '(주)아트라스비엑스'], 'LX하우시스': ['(주)엘지하우시스'], '오리온': ['(주)오리온'], '용평리조트': ['(주)용평리조트'], '우리은행': ['(주)우리은행'], '유니온': ['(주)유니온'], '유성티엔에스': ['(주)유성티엔에스'], '이마트': ['(주)이마트'], '이수페타시스': ['(주)이수페타시스'], '이수화학': ['(주)이수화학'], '인터플렉스': ['(주)인터플렉스'], '제이앤티씨': ['(주)제이앤티씨'], '종근당바이오': ['(주)종근당바이오'], '미래에셋드림스팩1호': ['미원스페셜티케미칼주식회사', nan, '(주)창해에탄올 익산사무소'], '체리부로': ['(주)체리부로'], '캐스텍코리아': ['(주)

In [77]:
## 유료 데이터 오류 직접 수정
name_match_dict['LG화학'] = ['주식회사 엘지화학']
name_match_dict['평화산업'] = ['평화산업(주)']
name_match_dict['한일시멘트'] = ['한일시멘트 주식회사']
name_match_dict['카프로'] = ['주식회사 카프로']
name_match_dict['한국조선해양'] = ['한국조선해양(주)']
name_match_dict['현대중공업'] = ['현대중공업주식회사', '현대중공업 주식회사']
name_match_dict['한일홀딩스'] = ['한일홀딩스(주)']

### b.비정제 기업명 변환 1: Fnguide 기업명(한글) 데이터 사용

In [78]:
## 전처리 완료 후 합치기
gas_non_code = pd.concat([gas_2011,gas_2012,gas_2013,gas_2014,gas_2015,gas_2021]) #사업자등록번호 없는 연도(2011~2015, 2021)
gas_code = pd.concat([gas_2016, gas_2017, gas_2018, gas_2019, gas_2020]).reset_index(drop=True) #사업자등록번호 있는 연도
gas_non_code['cor_name'] =  gas_non_code['non_cor_name']
gas_non_code.reset_index(drop=True,inplace=True)

In [79]:
## 비정제 기업명-정제 기업명으로 dict 구축 & replace
name_name_dict = df_to_dict(business_code, 'non_name','Name')
print(name_name_dict)
gas_non_code = gas_non_code.replace({'cor_name':name_name_dict})
gas_non_code #[gas_non_code['cor_name']!=gas_non_code['non_cor_name']] ## 변환된 기업만 출력

{'삼성전자(주)': '삼성전자', '(주)엘지에너지솔루션': 'LG에너지솔루션', '에스케이하이닉스(주)': 'SK하이닉스', '삼성바이오로직스(주)': '삼성바이오로직스', '삼성SDI(주)': '삼성SDI', '(주)엘지화학': 'LG화학', '현대자동차(주)': '현대차', '네이버(주)': 'NAVER', '기아(주)': '기아', '(주)셀트리온': '셀트리온', '포스코홀딩스(주)': 'POSCO홀딩스', '삼성물산(주)': '삼성물산', '(주)카카오': '카카오', '현대모비스(주)': '현대모비스', '(주)KB금융지주': 'KB금융', '(주)신한금융지주회사': '신한지주', '(주)포스코케미칼': '포스코케미칼', 'SK이노베이션(주)': 'SK이노베이션', '에스케이(주)': 'SK', '엘지전자(주)': 'LG전자', '삼성생명보험(주)': '삼성생명', '(주)케이티앤지': 'KT&G', '(주)엘지': 'LG', '고려아연(주)': '고려아연', '(주)하나금융지주': '하나금융지주', '한국전력공사(주)': '한국전력', '에스케이텔레콤(주)': 'SK텔레콤', '(주)카카오뱅크': '카카오뱅크', '(주)셀트리온헬스케어': '셀트리온헬스케어', '(주)에코프로비엠': '에코프로비엠', '삼성전기(주)': '삼성전기', '두산에너빌리티(주)': '두산에너빌리티', '에쓰-오일(주)': 'S-Oil', '에이치엠엠(주)': 'HMM', '삼성에스디에스(주)': '삼성에스디에스', '현대중공업(주)': '현대중공업', '(주)케이티': 'KT', '삼성화재해상보험(주)': '삼성화재', '한화솔루션(주)': '한화솔루션', '(주)크래프톤': '크래프톤', '(주)대한항공': '대한항공', '(주)엘지생활건강': 'LG생활건강', '(주)엔씨소프트': '엔씨소프트', '(주)우리금융지주': '우리금융지주', '(주)엘앤에프': '엘앤에프', '중소기업은행': '기업은행', 'LG이노텍(주)': 'LG이노텍', '현대글로비스(주)': 

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트
...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업 주식회사
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재 주식회사
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨 주식회사
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학 주식회사


In [80]:
## 변환되지 않은 기업 개수
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인:',len(gas_non_code[gas_non_code['non_cor_name']==gas_non_code['cor_name']])) ## 변환되지 X

사업자번호와 매칭이 안된 기업 개수확인: 3756


### c. 비정제 기업명 변환 2: 사업자등록번호 있는 연도 데이터에서 dict 구축 & 매칭

In [81]:
print(name_match_dict)

{'MH에탄올': ['(주)MH에탄올'], 'SIMPAC Metal': ['(주)SIMPAC METAL'], '강원랜드': ['(주)강원랜드'], '경방': ['(주)경방'], '국민은행': ['(주)국민은행'], '금비': ['(주)금비'], '대림제지': ['(주)대림제지'], '대우건설': ['(주)대우건설', '(주)대욱케스트'], '동방': ['(주)동방'], '동양고속': ['(주)동양고속'], '동원F&B': ['(주)동원F&B'], '두산': ['(주)두산'], '디에이피': ['(주)디에이피'], 'HL만도': ['(주)만도'], '태경비케이': ['주식회사 태경비케이', '(주)백광소재'], '벽산': ['(주)벽산'], '삼원강재': ['(주)삼원강재'], '삼천리': ['(주)삼천리'], '서원': ['(주)서원'], '서흥': ['(주)서흥'], '성광벤드': ['(주)성광벤드'], '세아베스틸지주': ['(주)세아베스틸'], '세아제강지주': ['(주)세아제강'], '세아특수강': ['(주)세아특수강'], '스타플렉스': ['(주)스타플렉스'], '신세계': ['(주)신세계'], '조흥은행': ['(주)신한은행'], '한국아트라스비엑스': ['(주)한국아트라스비엑스', '(주)아트라스비엑스'], 'LX하우시스': ['(주)엘지하우시스'], '오리온': ['(주)오리온'], '용평리조트': ['(주)용평리조트'], '우리은행': ['(주)우리은행'], '유니온': ['(주)유니온'], '유성티엔에스': ['(주)유성티엔에스'], '이마트': ['(주)이마트'], '이수페타시스': ['(주)이수페타시스'], '이수화학': ['(주)이수화학'], '인터플렉스': ['(주)인터플렉스'], '제이앤티씨': ['(주)제이앤티씨'], '종근당바이오': ['(주)종근당바이오'], '미래에셋드림스팩1호': ['미원스페셜티케미칼주식회사', nan, '(주)창해에탄올 익산사무소'], '체리부로': ['(주)체리부로'], '캐스텍코리아': ['(주)캐스텍코

In [82]:
## values에 해당되면 key로 치환
key_list = []
erorr_key_list = []

for non_name in gas_non_code['cor_name']:
  key = [k for k, v in name_match_dict.items() if non_name in v]
  if len(key) == 0:
    key_list.append(non_name)
  elif len(key) == 1:
    key_list.extend(key)
  else: #하나 values값에 keys가 두 개(ex.주식회사 XXX, XXX주식회사)
    erorr_key_list.append(key) #유료 데이터 오류 확인
    key_list.append(non_name)

gas_non_code['cor_name'] = key_list

In [83]:
print(erorr_key_list) ## 오류 없음
gas_non_code #[gas_non_code['cor_name']!=gas_non_code['non_cor_name']] ## 변환된 기업만 출력

[]


,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트
...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학


In [84]:
## 바뀐 이름(cor_name)에 대해 사업자번호 매칭(bsn_code) -> 이제부터는 bsn_code 누적 전처리
gas_non_code['bsn_code'] = gas_non_code['cor_name']
gas_non_code = gas_non_code.replace({'bsn_code':df_to_dict(business_code, 'Name','사업자등록번호')})
gas_non_code

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속,(주)경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업,(주)대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속,(주)대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트,(주)무주덕유산리조트
...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164


In [85]:
## 변환되지 않은 기업 개수
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인:',len([row for row in gas_non_code['bsn_code'] if r.search(row) is None])) ## 변환되지 X

사업자번호와 매칭이 안된 기업 개수확인: 2980


### d. 비정제 기업명 변환 3: 정규표현식 (주), (유), 주식회사 제거

In [86]:
gas_non_code['bsn_code'] = [re.sub("주식회사\s", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\s주식회사", "", row) for row in gas_non_code['bsn_code']] #앞 띄어쓰기
gas_non_code['bsn_code'] = [re.sub("주식회사", "", row) for row in gas_non_code['bsn_code']] #뒤 띄어쓰기

gas_non_code['bsn_code'] = [re.sub("유한회사\s", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\s유한회사", "", row) for row in gas_non_code['bsn_code']] #앞 띄어쓰기
gas_non_code['bsn_code'] = [re.sub("유한회사", "", row) for row in gas_non_code['bsn_code']] #뒤 띄어쓰기

gas_non_code['bsn_code'] = [re.sub("유한책임회사\s", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\s유한책임회사", "", row) for row in gas_non_code['bsn_code']] #앞 띄어쓰기
gas_non_code['bsn_code'] = [re.sub("유한책임회사", "", row) for row in gas_non_code['bsn_code']] #뒤 띄어쓰기

gas_non_code['bsn_code'] = [re.sub("\(주\)\s", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\s\(주\)", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\(주\)", "", row) for row in gas_non_code['bsn_code']]

gas_non_code['bsn_code'] = [re.sub("\(유\)\s", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\s\(유\)", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\(유\)", "", row) for row in gas_non_code['bsn_code']]

gas_non_code['bsn_code'] = [re.sub("㈜\s", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("\s㈜", "", row) for row in gas_non_code['bsn_code']]
gas_non_code['bsn_code'] = [re.sub("㈜", "", row) for row in gas_non_code['bsn_code']]

gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: row.strip()) ## 띄어쓰기 오류
# gas_non_code['bsn_code'] = [row.strip() for row in gas_non_code['bsn_code']]

In [87]:
## bsn_code에서 사업자번호 매칭 실패한 기업에 대해 위 전처리 이후, 다시 사업자번호 매칭(del_name)
print(df_to_dict(business_code, 'Name','사업자등록번호'))
gas_non_code['del_name']  = gas_non_code['bsn_code']
gas_non_code = gas_non_code.replace({'del_name':df_to_dict(business_code, 'Name','사업자등록번호')})
gas_non_code

{'삼성전자': '124-81-00998', 'LG에너지솔루션': '851-81-02050', 'SK하이닉스': '126-81-03725', '삼성바이오로직스': '131-86-27632', '삼성SDI': '124-81-31282', 'LG화학': '107-81-98139', '현대차': '101-81-09147', 'NAVER': '220-81-62517', '기아': '119-81-02316', '셀트리온': '133-81-23603', 'POSCO홀딩스': '506-81-00017', '삼성물산': '104-81-26728', '카카오': '120-81-47521', '현대모비스': '101-81-16406', 'KB금융': '201-86-08254', '신한지주': '104-81-64625', '포스코케미칼': '506-81-01452', 'SK이노베이션': '101-86-32319', 'SK': '116-81-02054', 'LG전자': '107-86-14075', '삼성생명': '104-81-26688', 'KT&G': '306-81-30866', 'LG': '116-81-18750', '고려아연': '211-81-11260', '하나금융지주': '107-86-71509', '한국전력': '120-82-00052', 'SK텔레콤': '104-81-37225', '카카오뱅크': '375-88-00197', '셀트리온헬스케어': '123-81-53473', '에코프로비엠': '638-86-00376', '삼성전기': '124-81-00979', '두산에너빌리티': '609-81-04684', 'S-Oil': '116-81-36743', 'HMM': '202-81-46197', '삼성에스디에스': '110-81-28774', '현대중공업': '252-87-01412', 'KT': '102-81-42945', '삼성화재': '202-81-45617', '한화솔루션': '202-81-16577', '크래프톤': '220-87-45316', '대한항공': '

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,del_name
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770,225-81-10770
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속,경기고속,경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업,대명레저산업,대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속,대원고속,대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트,무주덕유산리조트,무주덕유산리조트
...,...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896,578-87-00896
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929,198-87-00929
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070,880-87-01070
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164,175-88-01164


In [88]:
## 사업자번호와 매칭이 안된 기업 개수확인
print('사업자번호와 매칭이 안된 기업 개수확인:',len([row for row in gas_non_code['del_name'] if r.search(row) is None])) ## 변환되지 X

사업자번호와 매칭이 안된 기업 개수확인: 2924


### e. 비정제 기업명 변환 4: 정규표현식 한글 -> 영어

In [89]:
## 대기업 영어발음 사전
big_cor_eng_kor = {'SK': '에스케이', 'LG': '엘지', 'SH': '에스에이치', 'KT': '케이티',
       'DB':'디비', 'KG':'케이지', 'SPC': '에스피씨', 'SKC':'에스케이씨', 'OCI':'오씨아이'}

gas_non_code['eng_cor_name'] = gas_non_code['del_name']

for k, v in big_cor_eng_kor.items():
  gas_non_code['eng_cor_name'] = [re.sub(v, k, row) for row in gas_non_code['eng_cor_name']]

## del_name에서 사업자번호 매칭 실패한 기업에 대해 위 전처리 이후, 다시 사업자번호 매칭(eng_cor_name)
print(df_to_dict(business_code, 'Name','사업자등록번호'))
gas_non_code = gas_non_code.replace({'eng_cor_name':df_to_dict(business_code, 'Name','사업자등록번호')})
gas_non_code = gas_non_code.replace({'eng_cor_name':df_to_dict(business_code, 'Name','사업자등록번호')})

gas_non_code

{'삼성전자': '124-81-00998', 'LG에너지솔루션': '851-81-02050', 'SK하이닉스': '126-81-03725', '삼성바이오로직스': '131-86-27632', '삼성SDI': '124-81-31282', 'LG화학': '107-81-98139', '현대차': '101-81-09147', 'NAVER': '220-81-62517', '기아': '119-81-02316', '셀트리온': '133-81-23603', 'POSCO홀딩스': '506-81-00017', '삼성물산': '104-81-26728', '카카오': '120-81-47521', '현대모비스': '101-81-16406', 'KB금융': '201-86-08254', '신한지주': '104-81-64625', '포스코케미칼': '506-81-01452', 'SK이노베이션': '101-86-32319', 'SK': '116-81-02054', 'LG전자': '107-86-14075', '삼성생명': '104-81-26688', 'KT&G': '306-81-30866', 'LG': '116-81-18750', '고려아연': '211-81-11260', '하나금융지주': '107-86-71509', '한국전력': '120-82-00052', 'SK텔레콤': '104-81-37225', '카카오뱅크': '375-88-00197', '셀트리온헬스케어': '123-81-53473', '에코프로비엠': '638-86-00376', '삼성전기': '124-81-00979', '두산에너빌리티': '609-81-04684', 'S-Oil': '116-81-36743', 'HMM': '202-81-46197', '삼성에스디에스': '110-81-28774', '현대중공업': '252-87-01412', 'KT': '102-81-42945', '삼성화재': '202-81-45617', '한화솔루션': '202-81-16577', '크래프톤': '220-87-45316', '대한항공': '

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,del_name,eng_cor_name
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770,225-81-10770,225-81-10770
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속,경기고속,경기고속,경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업,대명레저산업,대명레저산업,대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속,대원고속,대원고속,대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트,무주덕유산리조트,무주덕유산리조트,무주덕유산리조트
...,...,...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896,578-87-00896,578-87-00896
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929,198-87-00929,198-87-00929
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070,880-87-01070,880-87-01070
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164,175-88-01164,175-88-01164


In [90]:
## 대기업 영어발음 사전으로 변환되지 않은 기업 개수
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인:',len([row for row in gas_non_code['eng_cor_name'] if r.search(row) is None])) ## 변환되지 X

사업자번호와 매칭이 안된 기업 개수확인: 2909


In [91]:
## 한글-영어발음 사전 및 기타
eng_kor = {'A': '에이', 'B': '비', 'C': '씨', 'D': '디',
       'E':'이', 'F':'에프', 'G': '지', 'H': '에이치',
       'I': '아이', 'J': '제이',  'K': '케이',  'L': '엘',
       'M':'엠', 'N':'엔', 'O': '오', 'P': '피',
       'Q': '큐', 'R': '알', 'S': '에스', 'T': '티',
       'U': '유', 'V': '브이','W':'더블유', 'X': '엑스',
       'Y':'와이', 'Z':'지', '&':'앤'}

## 정규표현식으로 전처리하고 사업자번호로 변환되지 않은 기업명 == Fnguide 미등록(재무정보가 없음)
gas_non_code['eng_cor_name2'] = gas_non_code['eng_cor_name']

for k, v in eng_kor.items():
  gas_non_code['eng_cor_name2'] = [re.sub(v, k, row) for row in gas_non_code['eng_cor_name2']]
print(df_to_dict(business_code, 'Name','사업자등록번호'))
gas_non_code = gas_non_code.replace({'eng_cor_name2':df_to_dict(business_code, 'Name','사업자등록번호')})
gas_non_code

gas_non_code

{'삼성전자': '124-81-00998', 'LG에너지솔루션': '851-81-02050', 'SK하이닉스': '126-81-03725', '삼성바이오로직스': '131-86-27632', '삼성SDI': '124-81-31282', 'LG화학': '107-81-98139', '현대차': '101-81-09147', 'NAVER': '220-81-62517', '기아': '119-81-02316', '셀트리온': '133-81-23603', 'POSCO홀딩스': '506-81-00017', '삼성물산': '104-81-26728', '카카오': '120-81-47521', '현대모비스': '101-81-16406', 'KB금융': '201-86-08254', '신한지주': '104-81-64625', '포스코케미칼': '506-81-01452', 'SK이노베이션': '101-86-32319', 'SK': '116-81-02054', 'LG전자': '107-86-14075', '삼성생명': '104-81-26688', 'KT&G': '306-81-30866', 'LG': '116-81-18750', '고려아연': '211-81-11260', '하나금융지주': '107-86-71509', '한국전력': '120-82-00052', 'SK텔레콤': '104-81-37225', '카카오뱅크': '375-88-00197', '셀트리온헬스케어': '123-81-53473', '에코프로비엠': '638-86-00376', '삼성전기': '124-81-00979', '두산에너빌리티': '609-81-04684', 'S-Oil': '116-81-36743', 'HMM': '202-81-46197', '삼성에스디에스': '110-81-28774', '현대중공업': '252-87-01412', 'KT': '102-81-42945', '삼성화재': '202-81-45617', '한화솔루션': '202-81-16577', '크래프톤': '220-87-45316', '대한항공': '

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,del_name,eng_cor_name,eng_cor_name2
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770,225-81-10770,225-81-10770,225-81-10770
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속,경기고속,경기고속,경기고속,경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업,대명레저산업,대명레저산업,대명레저산업,대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속,대원고속,대원고속,대원고속,대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트,무주덕유산리조트,무주덕유산리조트,무주덕유산리조트,무주덕U산리조트
...,...,...,...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896,578-87-00896,578-87-00896,578-87-00896
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929,198-87-00929,198-87-00929,198-87-00929
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070,880-87-01070,880-87-01070,880-87-01070
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164,175-88-01164,175-88-01164,175-88-01164


In [92]:
## 한글-영어발음 사전으로 변환되지 않은 기업 개수
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인:',len([row for row in gas_non_code['eng_cor_name2'] if r.search(row) is None])) ## 변환되지 X
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['eng_cor_name2'] if r.search(row) is None])))

사업자번호와 매칭이 안된 기업 개수확인: 2901
사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 986


In [93]:
match_bsn_list = []

# bsn_code 정리
for eng_row2, eng_row in zip(gas_non_code['eng_cor_name2'], gas_non_code['eng_cor_name']):
    r = re.compile("[0-9]+-[0-9]+-[0-9]+")
    if (r.search(eng_row2) is None) == False: #del_name 사업자번호와 매칭성공
      match_bsn_list.append(eng_row2)
    else:
      r = re.compile("^[a-zA-Z]+[a-zA-Z]$")
      if (r.search(eng_row2) is None) == False: #사업자번호와 매칭실패지만,기업명 영어단어
        match_bsn_list.append(eng_row2)
      else:
        match_bsn_list.append(eng_row)
gas_non_code['bsn_code'] = match_bsn_list

gas_non_code = gas_non_code.drop(['eng_cor_name','eng_cor_name2', 'del_name'], axis=1)
gas_non_code

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",(주)경기고속,경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,(주)대명레저산업,대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",(주)대원고속,대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,(주)무주덕유산리조트,무주덕유산리조트
...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164


In [94]:
# bsn_code와 매칭되는 기업명 cor_name 정리
gas_non_code['cor_name'] = gas_non_code['bsn_code']
print(df_to_dict(business_code, '사업자등록번호','Name'))
gas_non_code = gas_non_code.replace({'cor_name':df_to_dict(business_code, '사업자등록번호','Name')})
gas_non_code

{'124-81-00998': '삼성전자', '851-81-02050': 'LG에너지솔루션', '126-81-03725': 'SK하이닉스', '131-86-27632': '삼성바이오로직스', '124-81-31282': '삼성SDI', '107-81-98139': 'LG화학', '101-81-09147': '현대차', '220-81-62517': 'NAVER', '119-81-02316': '기아', '133-81-23603': '셀트리온', '506-81-00017': 'POSCO홀딩스', '202-81-45975': '삼성물산', '120-81-47521': '카카오', '101-81-16406': '현대모비스', '201-86-08254': 'KB금융', '104-81-64625': '신한지주', '506-81-01452': '포스코케미칼', '101-86-32319': 'SK이노베이션', '220-81-16503': 'SK', '107-86-14075': 'LG전자', '104-81-26688': '삼성생명', '306-81-30866': 'KT&G', '116-81-18750': 'LG', '211-81-11260': '고려아연', '107-86-71509': '하나금융지주', '120-82-00052': '한국전력', '104-81-37225': 'SK텔레콤', '375-88-00197': '카카오뱅크', '123-81-53473': '셀트리온헬스케어', '638-86-00376': '에코프로비엠', '124-81-00979': '삼성전기', '609-81-04684': '두산에너빌리티', '116-81-36743': 'S-Oil', '202-81-46197': 'HMM', '110-81-28774': '삼성에스디에스', '252-87-01412': '현대중공업', '102-81-42945': 'KT', '202-81-45617': '삼성화재', '202-81-16577': '한화솔루션', '220-87-45316': '크래프톤', '110-81-1

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",경기고속,경기고속
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,대명레저산업,대명레저산업
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",대원고속,대원고속
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,무주덕유산리조트,무주덕유산리조트
...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164


In [95]:
## 변환 오류 확인
r = re.compile("[가-힣][a-zA-Z]+[가-힣]")
# r = re.compile("[a-zA-Z][가-힣]+[a-zA-Z]")
print([row for row in gas_non_code['cor_name'] if r.search(row) is not None]) #'티KG에코머티리얼'

['용현BM양산공장', '삼성에스디에스수원ICT센터', '용현BM양산공장', '삼성에스디에스수원ICT센터', '미쓰이케미칼앤드SK씨폴리우레탄', '티KG에코머티리얼']


### g. 비정제 기업명 변환 5: 정규표현식 직접 수정 및 매칭

In [96]:
# gas_non_code[gas_non_code['cor_name']==gas_non_code['bsn_code']].groupby('bsn_code')['bsn_code'].count().sort_values(ascending=False)

In [97]:
gas_non_code['etc'] = ''

In [98]:
## 지역명 리스트
city = pd.read_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/한국행정구역분류.csv')
city_list = ['특별시$',' [가-힣]+', '시$', '군$', '구$', '읍$', '면$', '동$', '[0-9]+$']

for address in city_list:
  city['행정구역명'] = [re.sub(address, "", row) for row in city['행정구역명']]

address_list = list(set(city['행정구역명']))
address_list = [one for one in address_list if len(one)>1]
address_list.extend(['안동', '진해', '정읍', '인천', '영동', '음성', '대풍']) #직접 추가
print(address_list)

['대창', '세종특별자치', '삼덕', '고삼', '가리봉', '기흥', '양서', '도천', '운정', '포천', '장전', '팔복', '화양', '남원', '남현', '금호2.3가', '공릉', '서운', '울진', '원당', '신등', '생연', '춘궁', '개령', '회성', '구로', '파천', '산인', '속초', '광안', '감전', '풍호', '일직', '남해', '봉방', '문곡소도', '향남', '하빈', '덕적', '달서', '주산', '평창', '관저', '마두', '증도면병풍출장소', '어모', '청운효자', '월명', '가덕', '주문진', '구의', '길상', '신곡', '반곡', '진부', '마로', '갑천', '현북', '사직', '청림', '운곡', '복정', '회덕', '도안', '연기', '오류', '반여', '미천', '삼죽', '달산', '충청북도', '성정', '설악', '평내', '운천.신봉', '창릉', '부림', '광산', '덕천', '일신', '세곡', '신방', '안락', '반포본', '제주특별자치도', '서림', '분평', '소라', '부남', '근북', '문정', '진안', '고창', '오라', '강천', '나산', '배곧', '팔덕', '불국', '근덕', '우보', '동일', '연무', '흥천', '불광', '함안', '흥업', '진접', '세종대왕', '자금', '계남', '오송', '내곡', '일반성', '진례', '태안', '동인천', '태전', '화암', '의정부', '별량', '돌산읍죽포출장소', '용강', '고대', '거제', '장암', '원삼', '만촌', '삼산', '삼향', '이현', '은현', '시기', '광림', '본량', '구룡포', '온양', '동서학', '해제', '완월', '문내', '감천', '하망', '마산', '금호', '동량', '해미', '범천', '삼안', '부북', '죽변', '유달', '오창', '하일', '신길', '심곡', '김화', '중리', '삼화', '

In [99]:
## 지역+공장 기업명 삭제 후, 매칭
for address in address_list:
  gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: re.sub(address+'공장', "", row)) #'지역+공장'제거
  gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: re.sub(address+' ?[0-9]공장', "", row)) #'지역+공백+[0-9]+공장','지역+[0-9]+공장'제거
  gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: re.sub(address+' ?제?[0-9]공장', "", row))

gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: row.strip()) #공백제거

print(df_to_dict(business_code, 'Name','사업자등록번호'))
gas_non_code = gas_non_code.replace({'bsn_code':df_to_dict(business_code, 'Name','사업자등록번호')})

r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['bsn_code'] if r.search(row) is None])))

{'삼성전자': '124-81-00998', 'LG에너지솔루션': '851-81-02050', 'SK하이닉스': '126-81-03725', '삼성바이오로직스': '131-86-27632', '삼성SDI': '124-81-31282', 'LG화학': '107-81-98139', '현대차': '101-81-09147', 'NAVER': '220-81-62517', '기아': '119-81-02316', '셀트리온': '133-81-23603', 'POSCO홀딩스': '506-81-00017', '삼성물산': '104-81-26728', '카카오': '120-81-47521', '현대모비스': '101-81-16406', 'KB금융': '201-86-08254', '신한지주': '104-81-64625', '포스코케미칼': '506-81-01452', 'SK이노베이션': '101-86-32319', 'SK': '116-81-02054', 'LG전자': '107-86-14075', '삼성생명': '104-81-26688', 'KT&G': '306-81-30866', 'LG': '116-81-18750', '고려아연': '211-81-11260', '하나금융지주': '107-86-71509', '한국전력': '120-82-00052', 'SK텔레콤': '104-81-37225', '카카오뱅크': '375-88-00197', '셀트리온헬스케어': '123-81-53473', '에코프로비엠': '638-86-00376', '삼성전기': '124-81-00979', '두산에너빌리티': '609-81-04684', 'S-Oil': '116-81-36743', 'HMM': '202-81-46197', '삼성에스디에스': '110-81-28774', '현대중공업': '252-87-01412', 'KT': '102-81-42945', '삼성화재': '202-81-45617', '한화솔루션': '202-81-16577', '크래프톤': '220-87-45316', '대한항공': '

In [100]:
## 공장 기업명 삭제 후, 매칭
gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: re.sub('[0-9]?공장', "", row)) #'공장','[0-9]공장'제거
gas_non_code['bsn_code'] = gas_non_code['bsn_code'].apply(lambda row: row.strip()) #공백제거

print(df_to_dict(business_code, 'Name','사업자등록번호')) #사업자번호과 매칭
gas_non_code = gas_non_code.replace({'bsn_code':df_to_dict(business_code, 'Name','사업자등록번호')})

gas_non_code['cor_name'] = gas_non_code['bsn_code']
print(df_to_dict(business_code, '사업자등록번호','Name')) #사업자번호와 매칭되는 기업명
gas_non_code = gas_non_code.replace({'cor_name':df_to_dict(business_code, '사업자등록번호','Name')})

{'삼성전자': '124-81-00998', 'LG에너지솔루션': '851-81-02050', 'SK하이닉스': '126-81-03725', '삼성바이오로직스': '131-86-27632', '삼성SDI': '124-81-31282', 'LG화학': '107-81-98139', '현대차': '101-81-09147', 'NAVER': '220-81-62517', '기아': '119-81-02316', '셀트리온': '133-81-23603', 'POSCO홀딩스': '506-81-00017', '삼성물산': '104-81-26728', '카카오': '120-81-47521', '현대모비스': '101-81-16406', 'KB금융': '201-86-08254', '신한지주': '104-81-64625', '포스코케미칼': '506-81-01452', 'SK이노베이션': '101-86-32319', 'SK': '116-81-02054', 'LG전자': '107-86-14075', '삼성생명': '104-81-26688', 'KT&G': '306-81-30866', 'LG': '116-81-18750', '고려아연': '211-81-11260', '하나금융지주': '107-86-71509', '한국전력': '120-82-00052', 'SK텔레콤': '104-81-37225', '카카오뱅크': '375-88-00197', '셀트리온헬스케어': '123-81-53473', '에코프로비엠': '638-86-00376', '삼성전기': '124-81-00979', '두산에너빌리티': '609-81-04684', 'S-Oil': '116-81-36743', 'HMM': '202-81-46197', '삼성에스디에스': '110-81-28774', '현대중공업': '252-87-01412', 'KT': '102-81-42945', '삼성화재': '202-81-45617', '한화솔루션': '202-81-16577', '크래프톤': '220-87-45316', '대한항공': '

In [101]:
## 공장 전처리 이후에도 사업자 번호로 매칭이 안되는 '공장'기업명의 사업자 번호 '없음' 처리
r = re.compile("공장")
non_bsn_list = list(set([row for row in gas_non_code[gas_non_code['bsn_code']==gas_non_code['cor_name']]['non_cor_name'] if r.search(row) is not None]))
gas_non_code['bsn_code'][gas_non_code[gas_non_code['non_cor_name'].isin(non_bsn_list)].index] = '없음'
gas_non_code['etc'][gas_non_code[gas_non_code['non_cor_name'].isin(non_bsn_list)].index] = '공장'
gas_non_code[gas_non_code['etc']=='공장'].head()

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
52,(유)콘프로덕츠코리아 부평공장,2011,사업장,음식료품,NaN,NaN,콘프로덕츠코리아,없음,공장
53,(유)콘프로덕츠코리아 이천공장,2011,사업장,음식료품,NaN,NaN,콘프로덕츠코리아,없음,공장
77,오비맥주(주) 이천공장,2011,사업장,음식료품,NaN,NaN,오비맥주,없음,공장
90,(주)대승프론티어 군포공장,2011,사업장,유리 · 요업,"21,000",324,대승프론티어,없음,공장
97,(주)만도 원주공장,2011,사업장,자동차,NaN,NaN,만도,없음,공장


In [102]:
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['bsn_code'] if r.search(row) is None])))

사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 872


### h. 비정제 기업명 변환 6: 직접 대조해서, 기업재무정보 유무 확인 및 처리

  - 재무정보 있는지 기업명 키워드 Fn가이드 데이터에서 검색
  - 비슷한 이름이라도 같은 기업인지 사업자번호 구글 검색

#### 특수 건물 제외

In [103]:
## Fn가이드 기업정보등록이 되어있는가?
r = re.compile("호선") #'병원', '학교', '재단', '학원', '호선'관련 기업명의 재무정보 없음
[row for row in business_code['Name'] if r.search(row) is not None]

[]

In [104]:
## 특수 건물 필터
building_list = ['병원', '학교', '재단', '학원', '호선', '호텔', '리조트']

for build in building_list:
  r = re.compile(build+"$")
  build_list = [row for row in gas_non_code['bsn_code'] if r.search(row) is not None]
  gas_non_code['etc'][gas_non_code[gas_non_code['cor_name'].isin(build_list)].index] = build
  gas_non_code['bsn_code'][gas_non_code[gas_non_code['cor_name'].isin(build_list)].index] = '없음'

gas_non_code['etc'][gas_non_code[gas_non_code['cor_name']=='학교법인 성균관대학'].index] = '학교'
gas_non_code['bsn_code'][gas_non_code[gas_non_code['cor_name']=='학교법인 성균관대학'].index] = '없음'

gas_non_code[gas_non_code['bsn_code']=='없음'].head()

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,무주덕유산리조트,없음,리조트
5,(주)부산롯데호텔,2011,사업장,건물,"21,432",414,부산롯데호텔,없음,호텔
15,가톨릭대학교 서울성모병원,2011,사업장,건물,"42,037",812,가톨릭대학교 서울성모병원,없음,병원
16,건국대학교,2011,사업장,건물,"29,683",575,건국대학교,없음,학교
17,경북대학교,2011,업체,건물,"36,890",707,경북대학교,없음,학교


In [105]:
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['bsn_code'] if r.search(row) is None])))

사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 831


#### 행정구역 기업명 처리

In [106]:
city_cor_list = []
city_list = ['특별시$','광역시$', '시$', '군$', '구$', '읍$']

for address in city_list:
  r = re.compile(address)
  city_cor = [row for row in gas_non_code['bsn_code'] if r.search(row) is not None]
  city_cor_list.extend(city_cor)

city_cor_list = list(set(city_cor_list))
city_cor_list.remove('롯데엠알시')
city_cor_list.remove('롯데엠시시')
# city_cor_list.remove('오뚜기라면')
# city_cor_list.remove('신동')
# city_cor_list.remove('경동')

print(city_cor_list)
## 행정구역 기업명의 산업 카테고리 확인
gas_non_code.iloc[gas_non_code[gas_non_code['bsn_code'].isin(city_cor_list)].index].groupby('ctgy_detail')['ctgy_detail'].count()

['시흥시', '청주시', '진주시', '수원시', '여수시', '천안시', '춘천시', '안양시', '인천광역시', '군포시', '대구광역시', '김천시', '원주시', '의정부시', '남양주시', '순천시', '경기도 광주시', '용인시', '대전광역시', '포항시', '창원시', '부천시', '화성시', '경주시', '안산시', '아산시', '이천시', '충주시', '양주시', '강릉시', '제천시', '익산시', '전주시', '부산광역시', '포천시', '성남시', '경산시', '목포시', '서울특별시', '파주시', '거제시', '고양시', '구미시', '김해시', '광주광역시', '구리시', '군산시', '양산시', '울산광역시', '김포시', '광명시']


ctgy_detail
수도사업              1
지정외 폐기물처리업        1
지정외 폐기물처리업 등      3
폐기물             189
Name: ctgy_detail, dtype: int64

In [107]:
## 행정구역명 기업의 사업자번호 매칭 '없음' 처리
gas_non_code['etc'][gas_non_code[gas_non_code['bsn_code'].isin(city_cor_list)].index] = '지역산업 및 시설'
gas_non_code['bsn_code'][gas_non_code[gas_non_code['bsn_code'].isin(city_cor_list)].index] = '없음'
gas_non_code[gas_non_code['etc']=='지역산업 및 시설'].head(3)

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
465,광명시,2011,업체,폐기물,"77,236",283,광명시,없음,지역산업 및 시설
466,광주광역시,2011,업체,폐기물,"272,473","1,423",광주광역시,없음,지역산업 및 시설
468,대구광역시,2011,업체,폐기물,"176,238","1,420",대구광역시,없음,지역산업 및 시설


In [108]:
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['bsn_code'] if r.search(row) is None])))

사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 780


In [109]:
gas_non_code[gas_non_code['cor_name']==gas_non_code['bsn_code']].groupby('ctgy_detail')['ctgy_detail'].count()

ctgy_detail
건물                    175
건설                      2
공공·기타                   2
광업                     19
교통(여객)                 56
교통(철도)                 41
교통(해운)                 13
교통(화물)                 10
기계                     91
담배제조업                   1
디스플레이                   8
목재                     20
목재 및 나무제품 제조업;가구제외      1
반도체                    25
반도체.디스플레이.전기전자         75
발전 · 에너지               26
발전에너지                 105
비철금속                   92
산업                    227
산업단지                   33
석유화학                  279
섬유                     49
수도                     10
수도사업                    3
수송                     47
시멘트                    92
식료품 제조업                 6
요업                     15
유리                     31
유리 · 요업                66
음료제조업                   4
음식료품                   53
자동차                   110
전기전자                   19
전환                     53
정유                     20
제지                    132
조선                     29


#### 교통(여객, 철도, 항공, 해운, 화물)

In [110]:
## < 여객 >

# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("고속") #재무정보 있는 기업확인
print([row for row in business_code['Name'] if r.search(row) is not None])

gas_non_code[(gas_non_code['ctgy_detail']=='교통(여객)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])]

['천일고속', '동양고속']


,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",경기고속,경기고속,
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",대원고속,대원고속,
19,금호고속(주),2011,업체,교통(여객),"173,247","2,471",금호고속,금호고속,
495,(주)경기고속,2012,업체,교통(여객),"164,818","2,528",경기고속,경기고속,
505,(주)대원고속,2012,업체,교통(여객),"150,815","2,295",대원고속,대원고속,
625,금호고속(주),2012,업체,교통(여객),"176,085","3,027",금호고속,금호고속,
1069,(주)경기고속,2013,업체,교통(여객),167058,2604,경기고속,경기고속,
1072,(주)대원고속,2013,업체,교통(여객),154014,2387,대원고속,대원고속,
1094,금호고속(주),2013,업체,교통(여객),174007,2472,금호고속,금호고속,
2508,금호고속(주),2014,업체,교통(여객),175890,2498,금호고속,금호고속,


In [111]:
# 재무정보 없는 나머지는 전부 사업자번호'없음' 처리
gas_non_code['etc'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(여객)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '교통(여객)'
gas_non_code['bsn_code'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(여객)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '없음'
gas_non_code[gas_non_code['etc']=='교통(여객)'].head(3)

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",경기고속,없음,교통(여객)
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",대원고속,없음,교통(여객)
19,금호고속(주),2011,업체,교통(여객),"173,247","2,471",금호고속,없음,교통(여객)


In [112]:
## < 철도 >

# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("교통") #재무정보 있는 기업확인(철도X, 교통X, 메트로X)
print([row for row in business_code['Name'] if r.search(row) is not None])

# 재무정보 없는 나머지는 전부 사업자번호'없음' 처리
gas_non_code['etc'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(철도)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '교통(철도)'
gas_non_code['bsn_code'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(철도)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '없음'
gas_non_code[gas_non_code['etc']=='교통(철도)'].head(3)

['서울교통공사']


,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
20,대구도시철도공사,2011,업체,교통(철도),"78,584","1,515",대구도시철도공사,없음,교통(철도)
23,부산교통공사,2011,업체,교통(철도),"152,977","2,949",부산교통공사,없음,교통(철도)
28,서울메트로,2011,업체,교통(철도),"374,313","7,217",서울메트로,없음,교통(철도)


In [113]:
## < 항공 >

# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("항공")
print([row for row in business_code['Name'] if r.search(row) is not None])

gas_non_code[gas_non_code['ctgy_detail']=='교통(항공)'] # 모두 기업정보등록 있음

['대한항공', '한국항공우주', '아시아나항공', '제주항공', '티웨이항공', '하이즈항공']


,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
21,대한항공,2011,업체,교통(항공),"657,563","9,782",대한항공,110-81-14794,
31,아시아나항공,2011,업체,교통(항공),"297,679","4,372",아시아나항공,104-81-17480,
670,대한항공,2012,업체,교통(항공),"643,255","9,653",대한항공,110-81-14794,
802,아시아나항공,2012,업체,교통(항공),"297,668","4,376",아시아나항공,104-81-17480,
824,에어부산(주),2012,업체,교통(항공),"112,939","1,612",에어부산,605-81-84518,
3140,(주)진에어,2015,업체,교통(항공),162706,602,진에어,121-81-89086,
3141,티웨이항공,2015,업체,교통(항공),117222,1677,티웨이항공,101-81-94800,


In [114]:
## < 해운 >

# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("일신") #재무정보 있는 기업확인(해운X, 고속X, 라인X, 일신로지스틱스X, 일신해운X)
print([row for row in business_code['Name'] if r.search(row) is not None])
gas_non_code[gas_non_code['ctgy_detail']=='교통(해운)'] # 모두 기업정보등록 없음

# 전부 사업자번호'없음' 처리
gas_non_code['etc'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(해운)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '교통(해운)'
gas_non_code['bsn_code'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(해운)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '없음'
gas_non_code[gas_non_code['etc']=='교통(해운)'].head(3)

['일신방직', '일신석재', '일신바이오', '국일신동', '일신']


,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
2535,쌍용해운(주),2014,업체,교통(해운),87431,1215,쌍용해운,없음,교통(해운)
2536,씨월드고속훼리(주),2014,업체,교통(해운),52271,724,씨월드고속훼리,없음,교통(해운)
3142,쌍용해운(주),2015,업체,교통(해운),87384,1215,쌍용해운,없음,교통(해운)


In [115]:
## < 화물 >

# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("동성") #재무정보 있는 기업확인(신분X, 제때X, 화물X, 롯데X, 특수X, 물류X)
print([row for row in business_code['Name'] if r.search(row) is not None])
gas_non_code[gas_non_code['ctgy_detail']=='교통(화물)']

['동성화인텍', '동성케미컬', '동성제약', '동성화학', '동성하이켐', '동성반도체']


,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
2538,신분당선(주),2014,사업장,교통(화물),19703,405,신분당선,신분당선,
2543,CJ대한통운(주),2014,업체,교통(화물),203037,3269,CJ대한통운,110-81-05034,
2544,현대글로비스(주),2014,업체,교통(화물),71831,1056,현대글로비스,106-81-97118,
2545,(주)한진,2014,업체,교통(화물),121618,1808,한진,201-81-02823,
3137,신분당선(주),2015,사업장,교통(화물),20057,412,신분당선,신분당선,
3144,CJ대한통운(주),2015,업체,교통(화물),200848,3192,CJ대한통운,110-81-05034,
3145,(주)한진,2015,업체,교통(화물),128885,1922,한진,201-81-02823,
3146,현대글로비스(주),2015,업체,교통(화물),78000,1145,현대글로비스,106-81-97118,
3417,(주)삼우에프앤지,2021,사업장,교통(화물),"20,955",343,삼우에프앤지,삼우에프앤지,
3420,(주)세아엘앤에스,2021,사업장,교통(화물),"16,516",242,세아엘앤에스,세아엘앤에스,


In [116]:
# 재무정보 없는 나머지는 전부 사업자번호'없음' 처리
gas_non_code['etc'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(화물)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '교통(화물)'
gas_non_code['bsn_code'][gas_non_code[(gas_non_code['ctgy_detail']=='교통(화물)')&(gas_non_code['cor_name']==gas_non_code['bsn_code'])].index] = '없음'
gas_non_code[gas_non_code['etc']=='교통(화물)'].head(3)

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
2538,신분당선(주),2014,사업장,교통(화물),19703,405,신분당선,없음,교통(화물)
3137,신분당선(주),2015,사업장,교통(화물),20057,412,신분당선,없음,교통(화물)
3417,(주)삼우에프앤지,2021,사업장,교통(화물),"20,955",343,삼우에프앤지,없음,교통(화물)


In [117]:
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['bsn_code'] if r.search(row) is None])))

사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 732


In [118]:
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len([row for row in gas_non_code['bsn_code'] if r.search(row) is None]))

사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 2870


#### 자회사 처리

In [119]:
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
print('사업자번호와 매칭이 안된 기업 개수확인(중복 제외):',len(set([row for row in gas_non_code['bsn_code'] if r.search(row) is None])))

사업자번호와 매칭이 안된 기업 개수확인(중복 제외): 732


In [120]:
# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("삼성") #재무정보 있는 기업확인
print('Fnguide 재무정보 유무:', [row for row in business_code['Name'] if r.search(row) is not None])

r = re.compile("삼성")
print('탄소배출데이터에 존재하는 기업:', set([row for row in gas_non_code['bsn_code'] if r.search(row) is not None]))

Fnguide 재무정보 유무: ['삼성전자', '삼성바이오로직스', '삼성SDI', '삼성물산', '삼성생명', '삼성전기', '삼성에스디에스', '삼성화재', '삼성엔지니어링', '삼성중공업', '삼성카드', '삼성증권', '삼성출판사', '삼성제약', '삼성공조', '삼성스팩7호', '삼성스팩4호', '삼성스팩6호', '삼성머스트스팩5호', '삼성물산', '삼성건설', '삼성투신증권', '삼성수산', '삼성이미징']
탄소배출데이터에 존재하는 기업: {'삼성코닝어드밴스드 글라스', '삼성코닝어드밴스드글라스', '르노삼성자동차', '삼성에스디에스수원ICT센터', '삼성디스플레이', '삼성엘이디기흥사업장'}


In [121]:
# 재무정보 없는 나머지는 전부 사업자번호'없음' 처리 & 자회사의 모회사 이름 기록
firm_list = ["^현대", "^SK[가-힣]", "^포스코", "^한화", "^지에스", "쌍용", "삼성", "롯데"]

for firm in firm_list:
  r = re.compile(firm)
  gas_non_code['etc'][gas_non_code[gas_non_code['bsn_code'].isin([row for row in gas_non_code['bsn_code'] if r.search(row) is not None])].index] = firm
  gas_non_code['bsn_code'][gas_non_code[gas_non_code['bsn_code'].isin([row for row in gas_non_code['bsn_code'] if r.search(row) is not None])].index] = '없음'
  gas_non_code[gas_non_code['etc']==firm].head(3)

## (6) 사업자번호 있는 데이터와 없는 데이터 merge

In [122]:
## 사업자번호 있는 데이터

# Fn가이드 기업정보등록이 되어있는가?
r = re.compile("삼성") #재무정보 있는 기업확인
print('Fnguide 재무정보 유무:', [row for row in business_code['Name'] if r.search(row) is not None])

r = re.compile("삼성")
print('탄소배출데이터에 존재하는 기업:', set([row for row in gas_non_code['bsn_code'] if r.search(row) is not None]))

Fnguide 재무정보 유무: ['삼성전자', '삼성바이오로직스', '삼성SDI', '삼성물산', '삼성생명', '삼성전기', '삼성에스디에스', '삼성화재', '삼성엔지니어링', '삼성중공업', '삼성카드', '삼성증권', '삼성출판사', '삼성제약', '삼성공조', '삼성스팩7호', '삼성스팩4호', '삼성스팩6호', '삼성머스트스팩5호', '삼성물산', '삼성건설', '삼성투신증권', '삼성수산', '삼성이미징']
탄소배출데이터에 존재하는 기업: set()


In [123]:
# 사업자번호 매칭실패 기업명 모두 '없음'처리
r = re.compile("[0-9]+-[0-9]+-[0-9]+")
gas_non_code['etc'][gas_non_code[gas_non_code['bsn_code'].isin([row for row in gas_non_code['bsn_code'] if r.search(row) is None])].index] = '없음'
gas_non_code['bsn_code'][gas_non_code[gas_non_code['bsn_code'].isin([row for row in gas_non_code['bsn_code'] if r.search(row) is None])].index] = '없음'
gas_non_code[gas_non_code['etc']=='없음'].head(3)

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",경기고속,없음,없음
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,대명레저산업,없음,없음
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",대원고속,없음,없음


In [124]:
## 사업자번호 없는 데이터 전처리

print(df_to_dict(business_code, '사업자등록번호','Name'))
gas_code = gas_code.replace({'cor':df_to_dict(business_code, '사업자등록번호','Name')})

for inx in gas_code[gas_code['cor']==gas_code['bsn_code']].index:
  gas_code['cor'][inx] = '없음'

gas_code = gas_code.rename(columns={'cor':'cor_name'})
gas_code['etc'] = np.nan
gas_code = gas_code[gas_non_code.columns]

{'124-81-00998': '삼성전자', '851-81-02050': 'LG에너지솔루션', '126-81-03725': 'SK하이닉스', '131-86-27632': '삼성바이오로직스', '124-81-31282': '삼성SDI', '107-81-98139': 'LG화학', '101-81-09147': '현대차', '220-81-62517': 'NAVER', '119-81-02316': '기아', '133-81-23603': '셀트리온', '506-81-00017': 'POSCO홀딩스', '202-81-45975': '삼성물산', '120-81-47521': '카카오', '101-81-16406': '현대모비스', '201-86-08254': 'KB금융', '104-81-64625': '신한지주', '506-81-01452': '포스코케미칼', '101-86-32319': 'SK이노베이션', '220-81-16503': 'SK', '107-86-14075': 'LG전자', '104-81-26688': '삼성생명', '306-81-30866': 'KT&G', '116-81-18750': 'LG', '211-81-11260': '고려아연', '107-86-71509': '하나금융지주', '120-82-00052': '한국전력', '104-81-37225': 'SK텔레콤', '375-88-00197': '카카오뱅크', '123-81-53473': '셀트리온헬스케어', '638-86-00376': '에코프로비엠', '124-81-00979': '삼성전기', '609-81-04684': '두산에너빌리티', '116-81-36743': 'S-Oil', '202-81-46197': 'HMM', '110-81-28774': '삼성에스디에스', '252-87-01412': '현대중공업', '102-81-42945': 'KT', '202-81-45617': '삼성화재', '202-81-16577': '한화솔루션', '220-87-45316': '크래프톤', '110-81-1

In [125]:
gas_code = gas_code.dropna(subset=['year'], how='any', axis=0)
gas_code['year'] = gas_code['year'].astype(int)

In [126]:
## 연도순으로 합치기
gas_code_non_code = pd.concat([gas_code,gas_non_code],axis=0)

gas_df = pd.DataFrame()

for year in range(2011,2022):
  df_year = gas_code_non_code[gas_code_non_code['year']==year]

  gas_df = pd.concat([gas_df,df_year],axis=0)
gas_df

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
0,(주)강원랜드,2011,업체,건물,"70,829","1,301",강원랜드,225-81-10770,
1,(주)경기고속,2011,업체,교통(여객),"167,188","2,540",경기고속,없음,없음
2,(주)대명레저산업,2011,사업장,건물,"49,618",955,대명레저산업,없음,없음
3,(주)대원고속,2011,업체,교통(여객),"152,566","2,277",대원고속,없음,없음
4,(주)무주덕유산리조트,2011,사업장,건물,"27,636",465,무주덕유산리조트,없음,없음
...,...,...,...,...,...,...,...,...,...
4482,효성중공업 주식회사,2021,업체,산업,"54,101","1,132",효성중공업,578-87-00896,
4483,효성첨단소재 주식회사,2021,업체,산업,"202,512","4,210",효성첨단소재,198-87-00929,
4484,효성티앤씨 주식회사,2021,업체,산업,"365,131","8,403",효성티앤씨,880-87-01070,
4485,효성화학 주식회사,2021,업체,산업,"887,229","18,850",효성화학,175-88-01164,


In [127]:
gas_code_non_code['year'].unique()

array([2016, 2017, 2018, 2019, 2020, 2011, 2012, 2013, 2014, 2015, 2021])

In [128]:
gas_df['grngas'] = gas_df['grngas'].replace([' ******* ', '*****', '******'], np.nan)
gas_df['grngas'] = gas_df['grngas'].astype(str)
gas_df['grngas'] = [re.sub(",", "", row) for row in gas_df['grngas']]
gas_df['grngas'] = gas_df['grngas'].fillna(-11111).astype(float).replace({-11111: None})

gas_df['enrg'] = gas_df['enrg'].replace(['***', ' ******* ', '*****', '******', ' - '], np.nan)
gas_df['enrg'] = gas_df['enrg'].astype(str)
gas_df['enrg'] = [re.sub(",", "", row) for row in gas_df['enrg']]
gas_df['enrg'] = gas_df['enrg'].fillna(-11111).astype(float).replace({-11111: None})
gas_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9445 entries, 0 to 4486
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   non_cor_name  9445 non-null   object 
 1   year          9445 non-null   int64  
 2   ctgy          9445 non-null   object 
 3   ctgy_detail   9445 non-null   object 
 4   grngas        9348 non-null   float64
 5   enrg          9346 non-null   float64
 6   cor_name      9445 non-null   object 
 7   bsn_code      9441 non-null   object 
 8   etc           4487 non-null   object 
dtypes: float64(2), int64(1), object(6)
memory usage: 737.9+ KB


#### 중복 처리

In [129]:
## 같은 기업명으로 중복연도 존재하는지 확인
gas_df.groupby(['non_cor_name', 'year'])[['year']].count().rename(columns={'year':'count'}).sort_values(ascending=False, by='count').head(10)

count
non_cor_name year       
경원여객자동차(주)   2019      2
             2020      2
케이지피 주식회사    2011      2
주식회사 더블유티씨서울 2011      2
             2012      2
             2013      2
경원여객자동차(주)   2015      2
             2016      2
             2017      2
             2018      2

In [130]:
## 잘못된 사업자번호 삭제(재무정보X) 또는 변경
# 104-81-58180: 대욱케스트, 054-956-6363, 재무정보X -> drop
# 211-81-11275: 주식회사 영흥산업환경('영흥'과 다른 기업), 041-554-7811, 재무정보X -> drop
# 133-81-23689: 주식회사 대유글로벌('대유'과 다른 기업), 063-260-1407, 재무정보X -> drop
# 608-81-68647: 화영운수(주), 107-87-41846, 재무정보X -> drop
# 134-81-09623: 영특수츄레라(주), 113-81-75470, 재무정보X -> drop

gas_df = gas_df[~gas_df['non_cor_name'].isin(['(주)대욱케스트','주식회사 영흥산업환경','주식회사 대유글로벌','화영운수(주)', '평택시', '포천민자발전 주식회사', '영특수츄레라(주)'])].reset_index(drop=True)

In [131]:
## 잘못된 사업자번호 삭제(재무정보X) 또는 변경
# 125-81-00424: 영풍전자(주) 134-81-09623 -> 변경
# 506-81-05517: 포스코에너지(주) 104-81-48528 -> 변경
# 104-81-25980: 롯데쇼핑 주식회사 104-81-26067 -> 변경

for idx in list(gas_df[gas_df['non_cor_name']=='영풍전자(주)'].index):
  gas_df['bsn_code'][idx] = '134-81-09623'
  gas_df['cor_name'][idx] = '영풍전자'

for idx in list(gas_df[gas_df['non_cor_name']=='포스코에너지(주)'].index):
  gas_df['bsn_code'][idx] = '104-81-48528'
  gas_df['cor_name'][idx] = '포스코에너지'

for idx in list(gas_df[gas_df['non_cor_name']=='롯데쇼핑 주식회사'].index):
  gas_df['bsn_code'][idx] = '104-81-26067'
  gas_df['cor_name'][idx] = '롯데쇼핑'

for idx in list(gas_df[gas_df['non_cor_name']=='KPX 케미칼(주)'].index):
  gas_df['bsn_code'][idx] = '	110-81-33923'
  gas_df['cor_name'][idx] = 'KPX케미칼'
  gas_df['non_cor_name'][idx] = '케이피엑스케미칼(주)'

gas_df['bsn_code'] = gas_df['bsn_code'].replace('-', '없음')

In [132]:
## 같은 기업명으로 중복연도 존재하는지 확인
dup_bsn_year = gas_df.groupby(['bsn_code', 'year'])[['year']].count().rename(columns={'year':'count'}).sort_values(ascending=False, by='count')
bsn_code_list = dup_bsn_year[dup_bsn_year['count']>1].reset_index()['bsn_code'].unique()
bsn_code_list

array(['없음', '603-81-06646', '134-81-02858', '568-86-01173',
       '638-86-00376', '116-81-40780', '138-81-04799', '126-81-09409',
       '220-81-00490', '104-81-26728', '101-86-32319', '125-81-00201',
       '403-81-03729'], dtype=object)

In [133]:
def year_dup_drop(df, bsn_code_list):
  for bsn in bsn_code_list[1:]:
    df_bsn = df[df['bsn_code']==bsn].fillna(0)

    for year in df_bsn['year'].unique():
      df_year = df_bsn[df_bsn['year']==year]

      if len(df_year) > 1:
        non_cor_name = min(df_year['non_cor_name'].values, key=len)
        drop_non_cor_name= list(df_year[df_year['non_cor_name']!=non_cor_name]['non_cor_name'])

        grngas = np.sum(df_year['grngas'].values)
        enrg = np.sum(df_year['enrg'].values)

        ## drop & replace
        inx = df[(df['non_cor_name']==non_cor_name)&(df['year']==year)].index
        df.loc[inx, 'grngas'] = grngas
        df.loc[inx, 'enrg'] = enrg

        inx = df[(df['non_cor_name'].isin(drop_non_cor_name))&(df['year']==year)].index
        df = df.drop(inx,axis=0).reset_index(drop=True)

        # for i in inx:
        #   df.loc[i, 'non_cor_name'] = non_cor_name

  return df

gas_df = year_dup_drop(gas_df, bsn_code_list)

In [134]:
## 예외 데이터는 따로 처리
inx = gas_df[(gas_df['bsn_code']=='104-81-26728')&(gas_df['grngas'].duplicated())].index
gas_df = gas_df.drop(inx, axis=0).reset_index(drop=True)

In [135]:
## 확인
dup_bsn_year = gas_df.groupby(['bsn_code', 'year'])[['year']].count().rename(columns={'year':'count'}).sort_values(ascending=False, by='count')
bsn_code_list = dup_bsn_year[dup_bsn_year['count']>1].reset_index()['bsn_code'].unique()
bsn_code_list

array(['없음', '138-81-04799'], dtype=object)

## (7) 패널데이터(panel_df)와 탄소배출데이터(gas_df) 취합

In [136]:
gas_df['year'].unique()

array([2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021])

In [137]:
gas_df = gas_df[gas_df['bsn_code']!='없음']
gas_df

,non_cor_name,year,ctgy,ctgy_detail,grngas,enrg,cor_name,bsn_code,etc
0,(주)강원랜드,2011,업체,건물,70829.0,1301.0,강원랜드,225-81-10770,
7,(주)신세계,2011,업체,건물,135869.0,2626.0,신세계,201-81-32195,
8,(주)용평리조트,2011,사업장,건물,33940.0,649.0,용평리조트,226-81-24374,
9,(주)이마트,2011,업체,건물,498215.0,9588.0,이마트,206-86-50913,
21,대한항공,2011,업체,교통(항공),657563.0,9782.0,대한항공,110-81-14794,
...,...,...,...,...,...,...,...,...,...
9375,효성중공업 주식회사,2021,업체,산업,54101.0,1132.0,효성중공업,578-87-00896,
9376,효성첨단소재 주식회사,2021,업체,산업,202512.0,4210.0,효성첨단소재,198-87-00929,
9377,효성티앤씨 주식회사,2021,업체,산업,365131.0,8403.0,효성티앤씨,880-87-01070,
9378,효성화학 주식회사,2021,업체,산업,887229.0,18850.0,효성화학,175-88-01164,


In [138]:
panel_df['grngas'] = ''

for y, b, g in zip(gas_df['year'], gas_df['bsn_code'], gas_df['grngas']):
  inx = panel_df[(panel_df['year']==str(y))&(panel_df['bsn_code']==b)].index
  panel_df['grngas'][inx] = g

In [139]:
panel_gas = panel_df[panel_df['grngas']!='']
panel_gas

,code,cor_name,Accounting_Closing,year,cash/assets,liabilities/assets,liabilities-cash/assets,sales/cur_liabilities,cur_assets/cur_liabilities,retained_earnings/cur_liabilities,...,코드세세분류,코드소분류,코드중분류,주거래은행,최초상장일자,상장일자,unlist_date,unlist_reason,bsn_code,grngas
21,A005930,삼성전자,12,2011,0.094299,0.349721,0.255422,3.723047,1.613350,2.202731,...,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998,5296376.0
22,A005930,삼성전자,12,2012,0.103779,0.329104,0.225325,4.284904,1.859436,2.556529,...,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998,6004674.0
23,A005930,삼성전자,12,2013,0.076070,0.299236,0.223166,4.456608,2.158421,2.895822,...,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998,6299773.0
24,A005930,삼성전자,12,2014,0.073086,0.270523,0.197437,3.964439,2.213754,3.259313,...,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998,6772105.0
25,A005930,삼성전자,12,2015,0.093471,0.260632,0.167161,3.973107,2.471436,3.665769,...,264,264,26,우리은행,19750611,19750611,NaN,0,124-81-00998,6698062.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108505,A152330,코리아오토글라스,12.0,2015,0.023153,0.392785,0.369632,3.530969,1.148470,1.113879,...,2311,231,23,NaN,20151229,20151229,2020-12-18,해산 사유 발생,307-81-15780,91846.0
108506,A152330,코리아오토글라스,12.0,2016,0.040355,0.422813,0.382458,2.657964,1.050235,1.082504,...,2311,231,23,NaN,20151229,20151229,2020-12-18,해산 사유 발생,307-81-15780,84098.0
108507,A152330,코리아오토글라스,12.0,2017,0.037154,0.392555,0.355401,2.631108,1.294386,1.290646,...,2311,231,23,NaN,20151229,20151229,2020-12-18,해산 사유 발생,307-81-15780,78453.0
108508,A152330,코리아오토글라스,12.0,2018,0.014795,0.388654,0.373859,2.369583,1.465956,1.323281,...,2311,231,23,NaN,20151229,20151229,2020-12-18,해산 사유 발생,307-81-15780,80156.0


In [140]:
gas_df['ctgy_detail'].unique()

array(['건물', '교통(항공)', '음식료품', '목재', '유리 · 요업', '석유화학', '제지',
       '반도체.디스플레이.전기전자', '자동차', '섬유', '시멘트', '철강', '기계', '정유', '통신', '조선',
       '비철금속', '발전에너지', '폐기물', '항공', '유리', '요업', '전기전자', '반도체', '디스플레이',
       '발전 · 에너지', '집단에너지', '교통(여객)', '건설', '교통(화물)', '광업', '산업단지',
       '교통(철도)', '수도', '교통(해운)', '음료제조업', '식료품 제조업', '지정외 폐기물처리업', '산업',
       '지정외 폐기물처리업 등', '담배제조업', '수송', '전환', '공공기타'], dtype=object)

In [141]:
panel_gas.groupby('unlist_reason')['unlist_reason'].count()

unlist_reason
0                                        2591
감사의견 거절(감사범위 제한 및 계속기업 존속능력에 대한 불확실성)       6
감사의견 의견거절 2년 계속                            11
감사의견거절(감사범위제한)                              2
상장폐지신청('15.6.11)                           11
상장폐지신청(2005.10.14)                          7
상장폐지유예기간종료                                  1
신청에 의한 상장폐지                                23
영업활동정지 6월 계속                                6
자본전액잠식                                     13
지주회사(최대주주등)의 완전자회사화 등                      11
지주회사의 완전자회사화(지주회사 신규상장)                    15
타법인의 완전자회사로 편입                              6
피흡수합병                                      29
해산 사유 발생                                   95
Name: unlist_reason, dtype: int64

In [142]:
panel_df.groupby('unlist_reason')['unlist_reason'].count()

unlist_reason
'14사업연도 자본잠식률 50%이상 사유로 관리종목 지정 후 '15사업연도 반기 감사의견 거절                                                                                              64
'16사업연도 감사의견거절(계속기업 존속능력에 대한 불확실성) 및 '17사업연도 반기 감사의견거절(계속기업 존속능력에 대한 불확실성)                                                                        32
- 2회연속 자본잠식률 50% 이상 - 2009.3.31 자본잠식률 50% 이상으로 관리종목 지정 후, 최근 반기의 재무제표에 대한 감사인의 감사의견 거절                                                            32
- 주된 영업이 정지되어 코스닥시장상장규정 제38조제2항제1호(삭제 ‘09.12.2)의 상장폐지기준 해당 / 기업의 계속성 및 경영의 투명성 등을 종합적으로 고려한 결과 코스닥시장상장규정 제38조제2항제5호라목(신설 ‘09.12.2)의 상장폐지기준 해당     32
-2회연속 자본잠식률 50%이상 -2회연속 자기자본 10억원미만 -자본잠식률 50%이상 및 자기자본 10억원미만으로 관리종목 지정후 최근 반기 재무제표에 대한 외부감사인의 감사의견 의견거절                                         32
                                                                                                                                                ... 
회사정리절차 개시 신청 기각 결정                                                                          

In [143]:
# export
panel_gas.to_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/panel_gas.csv', encoding='utf-8-sig')

In [144]:
gas_df.to_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/gas_df.csv', encoding='utf-8-sig')

In [145]:
panel_df.to_csv('/content/drive/MyDrive/학교/연구 자료/기후금융/finance_data/panel_df.csv', encoding='utf-8-sig')